<a href="https://colab.research.google.com/github/OlegKret/---/blob/master/%22SimplexLibrary_ver1.0%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

from fractions import Fraction
from tabulate import tabulate

# Set NumPy print options
np.set_printoptions(precision=3, suppress=True, linewidth=100)

def create_tableau_for_maximization(num_vars, objective_coeffs, num_constraints,
                                    constraint_coeffs, constraint_types, b_values, is_maximization):
    """
    Creates the initial tableau for the simplex method, handling both maximization and minimization problems.

    Args:
        num_vars: The number of decision variables.
        objective_coeffs: Coefficients of the objective function.
        num_constraints: The number of constraints.
        constraint_coeffs: Coefficients of the constraints.
        constraint_types: Types of constraints ('<=', '>=', or '=').
        b_values: Right-hand side values of the constraints.
        is_maximization: Boolean indicating if it's a maximization problem (True) or minimization (False).

    Returns:
        tableau: The initial tableau.
        all_slack_vars: List of all slack/surplus/artificial variables.
        basis: List of basic variables.
        coeffs: List of coefficients in the objective function (excluding RHS).
    """
    # Ensure all RHS values are non-negative
    for i in range(num_constraints):
        if b_values[i] < 0:
            constraint_coeffs[i] = [-coeff for coeff in constraint_coeffs[i]]
            b_values[i] *= -1
            if constraint_types[i] == '<=':
                constraint_types[i] = '>='
            elif constraint_types[i] == '>=':
                constraint_types[i] = '<='

    # Count the number of artificial variables needed
    num_artificial_vars = sum(1 for t in constraint_types if t in ('>=', '='))

    # Calculate the total number of columns in the tableau
    num_slack_surplus_vars = sum(1 for t in constraint_types if t in ('<=', '>='))  # Slack and surplus variables
    total_columns = num_vars + num_slack_surplus_vars + num_artificial_vars + 1  # +1 for RHS column

    # Create an empty tableau
    tableau = np.zeros((num_constraints + 1, total_columns))

    all_slack_vars = []
    artificial_var_count = 0
    basis = []

    # Fill in the tableau with coefficients, adding slack/surplus/artificial variables as needed
    current_slack_surplus_index = num_vars
    current_artificial_index = num_vars + num_slack_surplus_vars

    for i in range(num_constraints):
        # Fill in decision variable coefficients
        tableau[i, :num_vars] = constraint_coeffs[i]

        if constraint_types[i] == '<=':
            # Add slack variable with coefficient of +1
            tableau[i, current_slack_surplus_index] = 1
            all_slack_vars.append(f's{i + 1}')
            basis.append(current_slack_surplus_index)  # Slack variable added to the basis
            current_slack_surplus_index += 1

        elif constraint_types[i] == '>=':
            # Add surplus variable with coefficient of -1
            tableau[i, current_slack_surplus_index] = -1
            all_slack_vars.append(f's{i + 1}')
            current_slack_surplus_index += 1

            # Add artificial variable with coefficient of +1
            tableau[i, current_artificial_index] = 1
            all_slack_vars.append(f'a{artificial_var_count + 1}')
            basis.append(current_artificial_index)  # Artificial variable added to the basis
            artificial_var_count += 1
            current_artificial_index += 1

        elif constraint_types[i] == '=':
            # Add artificial variable with coefficient of +1
            tableau[i, current_artificial_index] = 1
            all_slack_vars.append(f'a{artificial_var_count + 1}')
            basis.append(current_artificial_index)  # Artificial variable added to the basis
            artificial_var_count += 1
            current_artificial_index += 1

        # Set the RHS value
        tableau[i, -1] = b_values[i]

    # Initialize the coefficients list with zeros (excluding the RHS column)
    coeffs = [0] * (total_columns - 1)

    # Set the objective row coefficients in the coefficients list
    M = 1e6  # Big M value
    for col_idx in range(total_columns - 1):
        if col_idx < num_vars:
            coeffs[col_idx] = -objective_coeffs[col_idx] if is_maximization else objective_coeffs[col_idx]
        elif col_idx >= num_vars + num_slack_surplus_vars:
            # Artificial variable coefficients: -M for maximization, +M for minimization
            coeffs[col_idx] = -M if is_maximization else M
        else:
            coeffs[col_idx] = 0

    # Calculate the reduced costs for the objective row (Corrected)
    for j in range(total_columns - 1):
        original_coeff = coeffs[j]
        weighted_sum = 0
        for i in range(num_constraints):
            basic_var_index = basis[i]
            weighted_sum += coeffs[basic_var_index] * tableau[i, j]
        tableau[-1, j] = original_coeff - weighted_sum

    # Calculate the RHS value for the objective row (Corrected)
    rhs_value = 0
    for i in range(num_constraints):
        basic_var_index = basis[i]
        rhs_value += coeffs[basic_var_index] * tableau[i, -1]
    tableau[-1, -1] = rhs_value

    # Print initial basis information with values from the tableau
    print("Initial Basis Indices:", basis)
    print("Initial Basis Variables and Their Values:")
    for idx, basic_var_index in enumerate(basis):
        variable_name = all_slack_vars[basic_var_index - num_vars] if basic_var_index >= num_vars else f"x{basic_var_index + 1}"
        if variable_name.startswith('a'):
            variable_value = "M" if not is_maximization else "-M"
        elif variable_name.startswith('s'):
            variable_value = tableau[idx, -1]  # Taking value from the tableau's RHS
        else:
            variable_value = tableau[idx, -1]
        print(f"Variable {variable_name} (Index {basic_var_index}): Value = {variable_value}")

    return tableau, all_slack_vars, basis, coeffs










def identify_entering_column(tableau, is_maximization):
    """
    Identifies the entering column based on the objective row of the simplex tableau.

    Args:
        tableau: The current simplex tableau.
        is_maximization: Boolean indicating if it's a maximization problem (True) or minimization (False).

    Returns:
        entering_column: The index of the entering column.
    """
    # Extract the objective row (last row except for the RHS value)
    objective_row = tableau[-1, :-1]

    # For maximization, select the most positive coefficient
    if is_maximization:
        entering_column = np.argmax(objective_row)
        if objective_row[entering_column] <= 0:
            return None  # No positive coefficient found, optimal solution reached

    # For minimization, select the most negative coefficient
    else:
        entering_column = np.argmin(objective_row)
        if objective_row[entering_column] >= 0:
            return None  # No negative coefficient found, optimal solution reached

    return entering_column







def identify_leaving_row(tableau, entering_column):
    """
    Identifies the leaving row based on the minimum ratio test in the simplex tableau.

    Args:
        tableau: The current simplex tableau.
        entering_column: The index of the entering column.

    Returns:
        leaving_row: The index of the leaving row.
    """
    num_rows = tableau.shape[0] - 1  # Exclude the objective row
    ratios = []

    # Perform the minimum ratio test
    for i in range(num_rows):
        rhs_value = tableau[i, -1]
        entering_value = tableau[i, entering_column]

        # Only consider positive entering values to avoid division by zero or negative ratios
        if entering_value > 0:
            ratios.append(rhs_value / entering_value)
        else:
            ratios.append(np.inf)  # Invalid ratio for negative or zero coefficients

    # Find the row with the smallest non-negative ratio
    leaving_row = np.argmin(ratios)

    # If all ratios are infinite, no valid leaving row exists
    if ratios[leaving_row] == np.inf:
        return None  # Indicates unbounded solution

    return leaving_row



def locate_pivot_element(tableau, entering_column, leaving_row):
    """
    Locates the pivot element in the simplex tableau.

    Args:
        tableau: The current simplex tableau.
        entering_column: The index of the entering column.
        leaving_row: The index of the leaving row.

    Returns:
        pivot_element: The value of the pivot element.
    """
    pivot_element = tableau[leaving_row, entering_column]
    return pivot_element





def update_key_row(tableau, leaving_row, pivot_element):
    """
    Updates the key row by dividing every element in the row by the pivot element.

    Args:
        tableau: The current simplex tableau.
        leaving_row: The index of the leaving row (key row).
        pivot_element: The value of the pivot element.

    Returns:
        tableau: The updated tableau after normalizing the key row.
    """
    # Divide every element in the key row by the pivot element
    tableau[leaving_row, :] /= pivot_element
    return tableau




def update_non_key_rows(tableau, entering_column, leaving_row):
    """
    Updates the non-key rows to eliminate the entering variable, excluding the objective row.

    Args:
        tableau: The current simplex tableau.
        entering_column: The index of the entering column.
        leaving_row: The index of the leaving row (key row).

    Returns:
        tableau: The updated tableau after eliminating the entering variable from non-key rows.
    """
    num_rows = tableau.shape[0] - 1  # Exclude the objective row

    # Iterate over each non-key row except the objective row
    for i in range(num_rows):
        if i != leaving_row:
            key_ratio = tableau[i, entering_column] / tableau[leaving_row, entering_column]
            # Subtract the key ratio multiplied by the updated key row from the non-key row
            tableau[i, :] -= key_ratio * tableau[leaving_row, :]

            # Set the entering column value in the non-key row to zero explicitly (for precision handling)
            tableau[i, entering_column] = 0

    return tableau









def update_objective_row(tableau, coeffs, basis):
    """
    Updates the objective row in the simplex tableau based on the given basis.

    Args:
        tableau: The current simplex tableau.
        coeffs: List of coefficients from the original objective function.
        basis: List of current basic variables.

    Returns:
        tableau: The updated simplex tableau with the recalculated objective row.
    """
    num_cols = tableau.shape[1] - 1  # Exclude the RHS column
    objective_row = np.zeros(num_cols)

    # Calculate the updated objective row (Corrected to use basis coefficients)
    for j in range(num_cols):
        original_coeff = coeffs[j]
        weighted_sum = 0
        for i, basic_var_index in enumerate(basis):
            weighted_sum += coeffs[basic_var_index] * tableau[i, j]  # Use basis coefficient as weight
        objective_row[j] = original_coeff - weighted_sum

    # Update the tableau with the new objective row
    tableau[-1, :-1] = objective_row

    # Calculate the RHS value for the objective row
    rhs_value = 0
    for i, basic_var_index in enumerate(basis):
        rhs_value += coeffs[basic_var_index] * tableau[i, -1]
    tableau[-1, -1] = rhs_value

    return tableau







def simplex_iteration(tableau, coeffs, basis, is_maximization, all_vars, artificial_vars):
    """
    Performs one iteration of the simplex method.

    Args:
        tableau: The current simplex tableau.
        coeffs: List of coefficients from the original objective function.
        basis: List of current basic variables.
        is_maximization: Boolean indicating if it's a maximization problem (True) or minimization (False).
        all_vars: List of all variable names (decision, slack, surplus, artificial).
        artificial_vars: List of indices of artificial variables.

    Returns:
        tableau: The updated simplex tableau after one iteration.
        basis: The updated list of basic variables.
        all_vars: Updated list of all variable names.
        is_optimal: Boolean indicating if the optimal solution has been reached.
        is_unbounded: Boolean indicating if the solution is unbounded.
    """
    # 1. Identify the entering column
    entering_column = identify_entering_column(tableau, is_maximization)
    if entering_column is None:
        return tableau, basis, all_vars, True, False  # Optimal solution reached

    # 2. Identify the leaving row and perform minimum ratio test
    leaving_row = identify_leaving_row(tableau, entering_column)
    if leaving_row is None:
        return tableau, basis, all_vars, False, True  # Unbounded solution

    # Record the variable that leaves and the variable that enters
    leaving_variable = all_vars[basis[leaving_row]]
    entering_variable = all_vars[entering_column]

    # 3. Pivot operation
    # Locate the pivot element
    pivot_element = locate_pivot_element(tableau, entering_column, leaving_row)

    # Update the key row
    tableau = update_key_row(tableau, leaving_row, pivot_element)

    # Update non-key rows, excluding the objective row
    tableau = update_non_key_rows(tableau, entering_column, leaving_row)

    # 4. Update the basis
    basis[leaving_row] = entering_column

    # 5. Zero out the artificial variable if it leaves the basis
    if leaving_variable in artificial_vars:
        artificial_index = all_vars.index(leaving_variable)
        tableau[:, artificial_index] = 0  # Set all values in the column to zero
        coeffs[artificial_index] = 0  # Set the coefficient to zero

    # Update the objective row separately
    tableau = update_objective_row(tableau, coeffs, basis)

    # 6. Check for optimality condition
    if is_maximization:
        is_optimal = all(coef <= 0 for coef in tableau[-1, :-1])  # All coefficients in the objective row should be <= 0
    else:
        is_optimal = all(coef >= 0 for coef in tableau[-1, :-1])  # All coefficients in the objective row should be >= 0

    return tableau, basis, all_vars, is_optimal, False  # Return whether optimal or not, no unbounded condition







def extract_optimal_solution(tableau, basis, all_vars):
    """
    Extracts the optimal values of the decision variables, slack variables, and the final value of the objective function.

    Args:
        tableau: The final simplex tableau.
        basis: List of indices of the variables currently in the basis.
        all_vars: List of all variable names.

    Returns:
        optimal_values: Dictionary of optimal values for decision variables and slack variables.
        objective_value: The final optimal value of the objective function.
    """
    num_vars = len([var for var in all_vars if var.startswith('x')])  # Number of decision variables
    num_slack_vars = len([var for var in all_vars if var.startswith('s')])  # Number of slack variables
    optimal_values = {var: 0 for var in all_vars[:num_vars + num_slack_vars]}  # Initialize all decision and slack variables to zero

    # Extract values for decision variables and slack variables in the basis
    for i, basic_var_index in enumerate(basis):
        var_name = all_vars[basic_var_index]
        if var_name in optimal_values:
            optimal_values[var_name] = tableau[i, -1]  # RHS value

    # Extract the final value of the objective function
    objective_value = tableau[-1, -1]  # RHS value of the objective row

    return optimal_values, objective_value





# Helper function for the standard simplex maximization problem with only <= constraints
def simplex_method_leq(tableau, objective_coeffs):
    """
    Performs the simplex method for problems with only <= constraints.

    Args:
        tableau: The initial simplex tableau.
        objective_coeffs: The coefficients of the objective function.

    Returns:
        optimal_solution: The values of the decision variables at the optimal solution.
        optimal_value: The optimal value of the objective function.
    """
    iteration = 0

    while True:
        print(f"\nIteration {iteration}:")


        # Check if all coefficients in the objective row are non-negative
        if np.all(tableau[-1, :-1] >= 0):
            print("Optimal solution found!")
            break

        # Choose the entering column (most negative value in objective row)
        entering_col = np.argmin(tableau[-1, :-1])
        print(f"--> Entering column: {entering_col}")

        # Calculate the ratios for the leaving variable
        ratios = []
        for i in range(tableau.shape[0] - 1):
            if tableau[i, entering_col] > 0:
                ratios.append(tableau[i, -1] / tableau[i, entering_col])
            else:
                ratios.append(np.inf)

        if np.all(np.isinf(ratios)):
            print("Unbounded solution!")
            return None, None

        # Choose the leaving row (minimum ratio)
        leaving_row = np.argmin(ratios)
        print(f"--> Leaving row: {leaving_row}")

        # Perform the pivot operation
        pivot_element = tableau[leaving_row, entering_col]
        print(f"--> Pivot element: {pivot_element}")

        if abs(pivot_element) < 1e-10:
            print("Error: Pivot element is too small. Potential numerical instability.")
            return None, None

        # Normalize the pivot row
        tableau[leaving_row] /= pivot_element

        # Update the rest of the tableau
        for i in range(tableau.shape[0]):
            if i != leaving_row:
                row_factor = tableau[i, entering_col]
                tableau[i] -= row_factor * tableau[leaving_row]

        iteration += 1

    # Extract and print the optimal solution
    num_vars = len(objective_coeffs)
    basic_vars = [i for i in range(tableau.shape[1] - 1)
                  if np.count_nonzero(tableau[:-1, i]) == 1 and tableau[np.nonzero(tableau[:-1, i])[0][0], i] == 1]

    optimal_solution = np.zeros(num_vars)
    for var in basic_vars:
        if var < num_vars:
            optimal_solution[var] = tableau[np.nonzero(tableau[:-1, var])[0][0], -1]

    print("\nOptimal solution:")
    for i in range(num_vars):
        print(f"x{i + 1} = {optimal_solution[i]}")

    # Calculate the optimal value of the objective function
    optimal_value = sum(objective_coeffs[i] * optimal_solution[i] for i in range(num_vars))
    print("Optimal value:", optimal_value)

    return optimal_solution, optimal_value









def simplex_iteration_maximization(tableau, coeffs, basis, all_vars, artificial_vars):
    """
    Performs one iteration of the simplex method for maximization problems.

    Args:
        tableau: The current simplex tableau.
        coeffs: List of coefficients from the original objective function.
        basis: List of current basic variables.
        all_vars: List of all variable names (decision, slack, surplus, artificial).
        artificial_vars: List of indices of artificial variables.

    Returns:
        tableau: The updated simplex tableau after one iteration.
        basis: The updated list of basic variables.
        all_vars: Updated list of all variable names.
        is_optimal: Boolean indicating if the optimal solution has been reached.
        is_unbounded: Boolean indicating if the solution is unbounded.
    """
    # 1. Identify the entering column (select the most positive coefficient)
    entering_column = identify_entering_column_maximization(tableau)
    if entering_column is None:
        return tableau, basis, all_vars, True, False  # Optimal solution reached

    # 2. Identify the leaving row using the minimum ratio test
    leaving_row = identify_leaving_row_maximization(tableau, entering_column)
    if leaving_row is None:
        return tableau, basis, all_vars, False, True  # Unbounded solution

    # Ensure the leaving_row is within bounds
    if leaving_row >= len(basis):
        raise IndexError("Leaving row index is out of bounds for the basis list.")

    # Record the variable that leaves and the variable that enters
    leaving_variable = basis[leaving_row]
    entering_variable = entering_column

    # 3. Pivot operation
    # Locate the pivot element
    pivot_element = locate_pivot_element_maximization(tableau, entering_column, leaving_row)

    # Update the key row
    tableau = update_key_row_maximization(tableau, leaving_row, pivot_element)

    # Update non-key rows, excluding the objective row
    tableau = update_non_key_rows_maximization(tableau, entering_column, leaving_row)

    # 4. Update the basis
    basis[leaving_row] = entering_column

    # 5. Zero out the artificial variable if it leaves the basis
    if leaving_variable in artificial_vars:
        artificial_index = leaving_variable
        tableau[:, artificial_index] = 0  # Set all values in the column to zero
        coeffs[artificial_index] = 0  # Set the coefficient to zero

    # 6. Update the objective row for maximization
    tableau = update_objective_row_maximization(tableau, coeffs, basis)

    # 7. Check for optimality condition: iterate until all coefficients in the objective row are non-positive
    is_optimal = all(coef <= 0 for coef in tableau[-1, :-1])  # All coefficients in the objective row should be <= 0

    return tableau, basis, all_vars, is_optimal, False  # Return whether optimal or not, no unbounded condition




def update_key_row_maximization(tableau, leaving_row, pivot_element):
    """
    Updates the key row in the simplex tableau for maximization.

    Args:
        tableau: The current simplex tableau.
        leaving_row: The index of the leaving row (key row).
        pivot_element: The pivot element.

    Returns:
        tableau: The updated tableau after scaling the key row by the pivot element.
    """
    tableau[leaving_row, :] /= pivot_element
    return tableau







def print_basis_values(basis, tableau, all_vars, num_vars, num_slack_vars, artificial_vars, is_maximization):
    """
    Prints the current values of the variables in the basis with enhanced formatting and fractions.
    """
    M_value = "-M" if is_maximization else "M"

    print("\nBasis Variables and Their Values:")

    # Convert tableau elements to fractions before using tabulate
    tableau_fractions = [[Fraction(x).limit_denominator() for x in row] for row in tableau]

    # Create and print the tableau using tabulate
    headers = all_vars + ["RHS"]
    table = tabulate(tableau_fractions, headers=headers, tablefmt="fancy_grid")  # Use tableau_fractions here
    print(table)

    # Print the values of basic variables
    for idx, basis_var_index in enumerate(basis):
        variable_name = all_vars[basis_var_index]
        if basis_var_index in artificial_vars:
            value = M_value
        else:
            value = Fraction(tableau[idx, -1]).limit_denominator()
        print(f"Variable {variable_name} (Index {basis_var_index}): Value = {value}")






def set_coefficients_maximization_big_m(all_vars, objective_function, artificial_vars):
    """
    Sets the coefficients for maximization problems using Big M method.

    Args:
        all_vars: List of all variable names (decision, slack, surplus, artificial).
        objective_function: Coefficients of the decision variables from the original objective function.
        artificial_vars: List of artificial variables.

    Returns:
        coeffs: List of coefficients for the objective row including artificial and other variables.
    """
    M = 1e6  # Big M value
    coeffs = []

    for var in all_vars:
        if var in artificial_vars:
            coeffs.append(-M)  # Assign -M to artificial variables for maximization
        elif var in objective_function:
            coeffs.append(objective_function[var])  # Set corresponding coefficient from objective function
        else:
            coeffs.append(0)  # Assign 0 to slack and surplus variables

    return coeffs









def create_tableau_only_for_maximization(num_vars, objective_coeffs, num_constraints,
                                         constraint_coeffs, constraint_types, b_values):
    """
    Creates the initial tableau for the simplex method, specifically for maximization problems.

    Args:
        num_vars: The number of decision variables.
        objective_coeffs: Coefficients of the objective function.
        num_constraints: The number of constraints.
        constraint_coeffs: Coefficients of the constraints.
        constraint_types: Types of constraints ('<=', '>=', or '=').
        b_values: Right-hand side values of the constraints.

    Returns:
        tableau: The initial tableau.
        all_slack_vars: List of all slack/surplus/artificial variables.
        basis: List of basic variables.
        coeffs: List of coefficients in the objective function (excluding RHS).
    """
    # Ensure all RHS values are non-negative
    for i in range(num_constraints):
        if b_values[i] < 0:
            constraint_coeffs[i] = [-coeff for coeff in constraint_coeffs[i]]
            b_values[i] *= -1
            if constraint_types[i] == '<=':
                constraint_types[i] = '>='
            elif constraint_types[i] == '>=':
                constraint_types[i] = '<='

    # Count the number of artificial variables needed
    num_artificial_vars = sum(1 for t in constraint_types if t in ('>=', '='))

    # Calculate the total number of columns in the tableau
    num_slack_surplus_vars = sum(1 for t in constraint_types if t in ('<=', '>='))  # Slack and surplus variables
    total_columns = num_vars + num_slack_surplus_vars + num_artificial_vars + 1  # +1 for RHS column

    # Create an empty tableau
    # Create an empty tableau
    tableau = np.zeros((num_constraints + 1, total_columns))

    all_slack_vars = []
    artificial_var_count = 0
    basis = []

    # Fill in the tableau
    current_slack_surplus_index = num_vars
    current_artificial_index = num_vars + num_slack_surplus_vars

    for i in range(num_constraints):
        tableau[i, :num_vars] = constraint_coeffs[i]

        if constraint_types[i] == '<=':
            tableau[i, current_slack_surplus_index] = 1
            all_slack_vars.append(f's{i + 1}')
            basis.append(current_slack_surplus_index)
            current_slack_surplus_index += 1

        elif constraint_types[i] == '>=':
            tableau[i, current_slack_surplus_index] = -1
            all_slack_vars.append(f's{i + 1}')
            current_slack_surplus_index += 1

            tableau[i, current_artificial_index] = 1
            all_slack_vars.append(f'a{artificial_var_count + 1}')
            basis.append(current_artificial_index)
            artificial_var_count += 1
            current_artificial_index += 1

        elif constraint_types[i] == '=':
            tableau[i, current_artificial_index] = 1
            all_slack_vars.append(f'a{artificial_var_count + 1}')
            basis.append(current_artificial_index)
            artificial_var_count += 1
            current_artificial_index += 1

        tableau[i, -1] = b_values[i]

    # Initialize coefficients
    # Initialize coefficients
    coeffs = [-obj_coeff for obj_coeff in objective_coeffs]  # Negate objective function coefficients
    coeffs += [0] * num_slack_surplus_vars  # Add 0s for slack/surplus variables
    coeffs += [-M] * num_artificial_vars  # Add -M for artificial variables

    # Calculate the objective row (Corrected)
    objective_row = np.zeros(total_columns - 1)
    for j in range(total_columns - 1):
        original_coeff = coeffs[j]
        weighted_sum = 0
        for i, k in enumerate(basis):
            weighted_sum += coeffs[k] * tableau[i, j]
        objective_row[j] = original_coeff - weighted_sum

    tableau[-1, :-1] = objective_row

    # Calculate RHS for objective row
    rhs_value = 0
    for i, k in enumerate(basis):
        rhs_value += coeffs[k] * tableau[i, -1]
    tableau[-1, -1] = rhs_value

    # Print initial basis information (Corrected)
    print("Initial Basis Indices:", basis)
    print("Initial Basis Variables and Their Values:")
    for i, basic_var_index in enumerate(basis):
        variable_name = all_slack_vars[basic_var_index - num_vars] if basic_var_index >= num_vars else f"x{basic_var_index + 1}"
        if variable_name.startswith('a'):
            variable_value = "-M"  # Or you can directly use -M here
        elif variable_name.startswith('s'):
            variable_value = 0   # Slack variables start at 0
        else:
            variable_value = tableau[i, -1]
        print(f"Variable {variable_name} (Index {basic_var_index}): Value = {variable_value}")

    return tableau, all_slack_vars, basis, coeffs









def identify_entering_column_maximization(tableau):
    """
    Identifies the entering column based on the objective row of the simplex tableau for maximization problems.

    Args:
        tableau: The current simplex tableau.

    Returns:
        entering_column: The index of the entering column, or None if the optimal solution has been reached.
    """
    # Extract the objective row (last row except for the RHS value)
    objective_row = tableau[-1, :-1]

    # For maximization, select the most positive coefficient
    entering_column = np.argmax(objective_row)
    if objective_row[entering_column] <= 0:
        return None  # No positive coefficient found, optimal solution reached

    return entering_column








def identify_leaving_row_maximization(tableau, entering_column):
    """
    Identifies the leaving row based on the minimum ratio test in the simplex tableau for maximization problems.

    Args:
        tableau: The current simplex tableau.
        entering_column: The index of the entering column.

    Returns:
        leaving_row: The index of the leaving row, or None if the solution is unbounded.
    """
    num_rows = tableau.shape[0] - 1  # Exclude the objective row
    ratios = []

    # Perform the minimum ratio test
    for i in range(num_rows):
        rhs_value = tableau[i, -1]
        entering_value = tableau[i, entering_column]

        # Only consider positive entering values to avoid division by zero or negative ratios
        if entering_value > 0:
            ratios.append(rhs_value / entering_value)
        else:
            ratios.append(np.inf)  # Invalid ratio for negative or zero coefficients

    # Find the row with the smallest non-negative ratio
    leaving_row = np.argmin(ratios)

    # If all ratios are infinite, no valid leaving row exists
    if ratios[leaving_row] == np.inf:
        return None  # Indicates unbounded solution

    return leaving_row







def locate_pivot_element_maximization(tableau, entering_column, leaving_row):
    """
    Locates the pivot element in the simplex tableau for maximization problems.

    Args:
        tableau: The current simplex tableau.
        entering_column: The index of the entering column.
        leaving_row: The index of the leaving row.

    Returns:
        pivot_element: The value of the pivot element.
    """
    pivot_element = tableau[leaving_row, entering_column]
    return pivot_element













def update_non_key_rows_maximization(tableau, entering_column, leaving_row):
    """
    Updates the non-key rows to eliminate the entering variable for maximization problems, excluding the objective row.

    Args:
        tableau: The current simplex tableau.
        entering_column: The index of the entering column.
        leaving_row: The index of the leaving row (key row).

    Returns:
        tableau: The updated tableau after eliminating the entering variable from non-key rows.
    """
    num_rows = tableau.shape[0] - 1  # Exclude the objective row

    # Iterate over each non-key row except the objective row
    for i in range(num_rows):
        if i != leaving_row:
            key_ratio = tableau[i, entering_column] / tableau[leaving_row, entering_column]
            # Subtract the key ratio multiplied by the updated key row from the non-key row
            tableau[i, :] -= key_ratio * tableau[leaving_row, :]

            # Set the entering column value in the non-key row to zero explicitly (for precision handling)
            tableau[i, entering_column] = 0

    return tableau






def update_objective_row_maximization(tableau, coeffs, basis):
    """
    Updates the objective row in the simplex tableau for maximization problems based on the given basis.

    Args:
        tableau: The current simplex tableau.
        coeffs: List of coefficients from the original objective function.
        basis: List of current basic variables.

    Returns:
        tableau: The updated simplex tableau with the recalculated objective row.
    """
    num_cols = tableau.shape[1] - 1  # Exclude the RHS column
    objective_row = np.zeros(num_cols)

    # Calculate the updated objective row using basis coefficients (for maximization)
    for j in range(num_cols):
        original_coeff = coeffs[j]
        weighted_sum = 0
        for i, basic_var_index in enumerate(basis):
            weighted_sum += coeffs[basic_var_index] * tableau[i, j]  # Use basis coefficient as weight
        objective_row[j] = original_coeff - weighted_sum

    # Update the tableau with the new objective row
    tableau[-1, :-1] = objective_row

    # Calculate the RHS value for the objective row
    rhs_value = 0
    for i, basic_var_index in enumerate(basis):
        rhs_value += coeffs[basic_var_index] * tableau[i, -1]
    tableau[-1, -1] = rhs_value

    return tableau


In [ ]:
# Example usage
num_vars = 2
objective_coeffs = [1, 4]
num_constraints = 3
constraint_coeffs = [
    [1, 1],
    [1, 3],
    [1, 2]
]
constraint_types = ['>=', '>=', '<=']
b_values = [1000, 4000, 3500]
is_maximization = False

tableau, all_slack_vars, basis, coeffs = create_tableau_for_maximization(num_vars, objective_coeffs, num_constraints,
                                                                         constraint_coeffs, constraint_types, b_values, is_maximization)

print("Initial Tableau:")
print(tableau)
print("All Slack/Surplus/Artificial Variables:", all_slack_vars)
print("Initial Basis:", basis)
print("Objective Function Coefficients (Reduced Costs):", coeffs)

Initial Tableau:
[[ 1.000000e+00  1.000000e+00 -1.000000e+00  0.000000e+00  0.000000e+00
   1.000000e+00  0.000000e+00  1.000000e+03]
 [ 1.000000e+00  3.000000e+00  0.000000e+00 -1.000000e+00  0.000000e+00
   0.000000e+00  1.000000e+00  4.000000e+03]
 [ 1.000000e+00  2.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00
   0.000000e+00  0.000000e+00  3.500000e+03]
 [-1.999999e+06 -3.999996e+06  1.000000e+06  1.000000e+06  0.000000e+00
   0.000000e+00  0.000000e+00  5.000000e+09]]
All Slack/Surplus/Artificial Variables: ['s1', 'a1', 's2', 'a2', 's3']
Initial Basis: [5, 6, 4]
Objective Function Coefficients (Reduced Costs): [1, 4, 0, 0, 0, 1000000.0, 1000000.0]


In [ ]:
import unittest
import numpy as np

# Assuming create_tableau_for_maximization function has been defined as per previous discussion

# Define the unit tests
class TestCreateTableauForMaximization(unittest.TestCase):

    def test_single_variable_le_constraint(self):
        """Test case with a single decision variable and a <= constraint."""
        num_vars = 1
        objective_coeffs = [2]
        num_constraints = 1
        constraint_coeffs = [[3]]
        constraint_types = ['<=']
        b_values = [4]
        is_maximization = False

        tableau, _, basis, coeffs = create_tableau_for_maximization(
            num_vars, objective_coeffs, num_constraints, constraint_coeffs, constraint_types, b_values, is_maximization
        )

        # Assert basis is correctly set
        self.assertEqual(basis, [1])
        # Assert RHS is correct
        self.assertEqual(tableau[0, -1], 4)
        # Assert coefficient for slack variable is 0
        self.assertEqual(coeffs[1], 0)

    def test_single_variable_ge_constraint(self):
        """Test case with a single decision variable and a >= constraint."""
        num_vars = 1
        objective_coeffs = [1]
        num_constraints = 1
        constraint_coeffs = [[2]]
        constraint_types = ['>=']
        b_values = [5]
        is_maximization = False

        tableau, _, basis, coeffs = create_tableau_for_maximization(
            num_vars, objective_coeffs, num_constraints, constraint_coeffs, constraint_types, b_values, is_maximization
        )

        # Assert artificial variable is added to the basis
        self.assertEqual(basis, [2])
        # Assert RHS is correct
        self.assertEqual(tableau[0, -1], 5)
        # Assert coefficient for artificial variable is M
        self.assertEqual(coeffs[2], 1e6)

    def test_two_variables_equal_constraint(self):
        """Test case with two decision variables and an = constraint."""
        num_vars = 2
        objective_coeffs = [3, 1]
        num_constraints = 1
        constraint_coeffs = [[1, 1]]
        constraint_types = ['=']
        b_values = [6]
        is_maximization = True

        tableau, _, basis, coeffs = create_tableau_for_maximization(
            num_vars, objective_coeffs, num_constraints, constraint_coeffs, constraint_types, b_values, is_maximization
        )

        # Assert artificial variable is added to the basis
        self.assertEqual(basis, [2])
        # Assert RHS is correct
        self.assertEqual(tableau[0, -1], 6)
        # Assert coefficient for artificial variable is -M
        self.assertEqual(coeffs[2], -1e6)

    def test_mixed_constraints(self):
        """Test case with mixed constraints (<=, >=, =)."""
        num_vars = 2
        objective_coeffs = [1, 4]
        num_constraints = 3
        constraint_coeffs = [
            [1, 2],
            [3, 1],
            [2, 3]
        ]
        constraint_types = ['<=', '>=', '=']
        b_values = [10, 15, 20]
        is_maximization = False

        tableau, _, basis, coeffs = create_tableau_for_maximization(
            num_vars, objective_coeffs, num_constraints, constraint_coeffs, constraint_types, b_values, is_maximization
        )

        # Assert basis contains correct slack and artificial variables
        self.assertEqual(basis, [2, 4, 5])
        # Assert RHS is correct
        self.assertEqual(tableau[0, -1], 10)
        self.assertEqual(tableau[1, -1], 15)
        self.assertEqual(tableau[2, -1], 20)

    def test_three_variables_all_le_constraints(self):
        """Test case with three decision variables and all <= constraints."""
        num_vars = 3
        objective_coeffs = [2, 3, 1]
        num_constraints = 2
        constraint_coeffs = [
            [1, 1, 1],
            [2, 2, 2]
        ]
        constraint_types = ['<=', '<=']
        b_values = [5, 10]
        is_maximization = True

        tableau, _, basis, coeffs = create_tableau_for_maximization(
            num_vars, objective_coeffs, num_constraints, constraint_coeffs, constraint_types, b_values, is_maximization
        )

        # Assert basis contains slack variables
        self.assertEqual(basis, [3, 4])
        # Assert coefficient for slack variables is 0
        self.assertEqual(coeffs[3], 0)
        self.assertEqual(coeffs[4], 0)

    def test_three_variables_all_ge_constraints(self):
        """Test case with three decision variables and all >= constraints."""
        num_vars = 3
        objective_coeffs = [4, 5, 6]
        num_constraints = 2
        constraint_coeffs = [
            [3, 2, 1],
            [1, 4, 3]
        ]
        constraint_types = ['>=', '>=']
        b_values = [8, 12]
        is_maximization = False

        tableau, _, basis, coeffs = create_tableau_for_maximization(
            num_vars, objective_coeffs, num_constraints, constraint_coeffs, constraint_types, b_values, is_maximization
        )

        # Assert artificial variables are added to the basis
        self.assertEqual(basis, [4, 5])
        # Assert coefficient for artificial variables is M
        self.assertEqual(coeffs[4], 1e6)
        self.assertEqual(coeffs[5], 1e6)

    def test_no_constraints(self):
        """Test case with no constraints (only objective function)."""
        num_vars = 2
        objective_coeffs = [1, 3]
        num_constraints = 0
        constraint_coeffs = []
        constraint_types = []
        b_values = []
        is_maximization = True

        tableau, _, basis, coeffs = create_tableau_for_maximization(
            num_vars, objective_coeffs, num_constraints, constraint_coeffs, constraint_types, b_values, is_maximization
        )

        # Assert tableau has only the objective row
        self.assertEqual(tableau.shape, (1, num_vars + 1))
        # Assert objective coefficients are set correctly
        self.assertEqual(tableau[-1, 0], -1)
        self.assertEqual(tableau[-1, 1], -3)

# Run the tests in Google Colab
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


....F..
FAIL: test_three_variables_all_ge_constraints (__main__.TestCreateTableauForMaximization)
Test case with three decision variables and all >= constraints.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-43-7c2847f5ca69>", line 138, in test_three_variables_all_ge_constraints
    self.assertEqual(basis, [4, 5])
AssertionError: Lists differ: [5, 6] != [4, 5]

First differing element 0:
5
4

- [5, 6]
+ [4, 5]

----------------------------------------------------------------------
Ran 7 tests in 0.015s

FAILED (failures=1)


In [ ]:
# Example usage
import numpy as np

# Assume the tableau is already created, e.g., after the first iteration
# Sample tableau from previous example
tableau = np.array([
    [3.0, 2.0, 1.0, -1.0, 0.0, 1.0, 0.0, 8.0],
    [1.0, 4.0, 3.0, 0.0, -1.0, 0.0, 1.0, 12.0],
    [-3.999996e6, -5.999995e6, -3.999994e6, 1e6, 1e6, 0.0, 0.0, 2.0e7]
])

is_maximization = False

# Find the entering column
entering_column = identify_entering_column(tableau, is_maximization)
print(f"Entering Column: {entering_column}")


Entering Column: 1


In [ ]:
# Example usage
# Sample tableau from the previous example
tableau = np.array([
    [3.0, 2.0, 1.0, -1.0, 0.0, 1.0, 0.0, 8.0],
    [1.0, 4.0, 3.0, 0.0, -1.0, 0.0, 1.0, 12.0],
    [-3.999996e6, -5.999995e6, -3.999994e6, 1e6, 1e6, 0.0, 0.0, 2.0e7]
])

is_maximization = False

# Find the entering column
entering_column = identify_entering_column(tableau, is_maximization)
print(f"Entering Column: {entering_column}")

# Find the leaving row
if entering_column is not None:
    leaving_row = identify_leaving_row(tableau, entering_column)
    print(f"Leaving Row: {leaving_row}")
else:
    print("No entering column found; optimal solution reached.")



Entering Column: 1
Leaving Row: 1


In [ ]:
# Example usage
# Sample tableau from the previous example
tableau = np.array([
    [3.0, 2.0, 1.0, -1.0, 0.0, 1.0, 0.0, 8.0],
    [1.0, 4.0, 3.0, 0.0, -1.0, 0.0, 1.0, 12.0],
    [-3.999996e6, -5.999995e6, -3.999994e6, 1e6, 1e6, 0.0, 0.0, 2.0e7]
])

is_maximization = False

# Find the entering column
entering_column = identify_entering_column(tableau, is_maximization)
print(f"Entering Column: {entering_column}")

# Find the leaving row
if entering_column is not None:
    leaving_row = identify_leaving_row(tableau, entering_column)
    if leaving_row is not None:
        print(f"Leaving Row: {leaving_row}")

        # Locate the pivot element
        pivot_element = locate_pivot_element(tableau, entering_column, leaving_row)
        print(f"Pivot Element: {pivot_element}")
    else:
        print("No leaving row found; solution is unbounded.")
else:
    print("No entering column found; optimal solution reached.")



Entering Column: 1
Leaving Row: 1
Pivot Element: 4.0


In [ ]:
# Example usage
# Sample tableau from the previous example
tableau = np.array([
    [3.0, 2.0, 1.0, -1.0, 0.0, 1.0, 0.0, 8.0],
    [1.0, 4.0, 3.0, 0.0, -1.0, 0.0, 1.0, 12.0],
    [-3.999996e6, -5.999995e6, -3.999994e6, 1e6, 1e6, 0.0, 0.0, 2.0e7]
])

is_maximization = False

# Find the entering column
entering_column = identify_entering_column(tableau, is_maximization)
print(f"Entering Column: {entering_column}")

# Find the leaving row
if entering_column is not None:
    leaving_row = identify_leaving_row(tableau, entering_column)
    if leaving_row is not None:
        print(f"Leaving Row: {leaving_row}")

        # Locate the pivot element
        pivot_element = locate_pivot_element(tableau, entering_column, leaving_row)
        print(f"Pivot Element: {pivot_element}")

        # Update the key row
        tableau = update_key_row(tableau, leaving_row, pivot_element)
        print("Updated Tableau after Key Row Normalization:")
        print(tableau)
    else:
        print("No leaving row found; solution is unbounded.")
else:
    print("No entering column found; optimal solution reached.")


Entering Column: 1
Leaving Row: 1
Pivot Element: 4.0
Updated Tableau after Key Row Normalization:
[[ 3.000000e+00  2.000000e+00  1.000000e+00 -1.000000e+00  0.000000e+00
   1.000000e+00  0.000000e+00  8.000000e+00]
 [ 2.500000e-01  1.000000e+00  7.500000e-01  0.000000e+00 -2.500000e-01
   0.000000e+00  2.500000e-01  3.000000e+00]
 [-3.999996e+06 -5.999995e+06 -3.999994e+06  1.000000e+06  1.000000e+06
   0.000000e+00  0.000000e+00  2.000000e+07]]


In [ ]:
# Example usage
# Sample tableau from the previous example
tableau = np.array([
    [3.0, 2.0, 1.0, -1.0, 0.0, 1.0, 0.0, 8.0],
    [1.0, 4.0, 3.0, 0.0, -1.0, 0.0, 1.0, 12.0],
    [-3.999996e6, -5.999995e6, -3.999994e6, 1e6, 1e6, 0.0, 0.0, 2.0e7]
])

is_maximization = False

# Find the entering column
entering_column = identify_entering_column(tableau, is_maximization)
print(f"Entering Column: {entering_column}")

# Find the leaving row
if entering_column is not None:
    leaving_row = identify_leaving_row(tableau, entering_column)
    if leaving_row is not None:
        print(f"Leaving Row: {leaving_row}")

        # Locate the pivot element
        pivot_element = locate_pivot_element(tableau, entering_column, leaving_row)
        print(f"Pivot Element: {pivot_element}")

        # Update the key row
        tableau = update_key_row(tableau, leaving_row, pivot_element)
        print("Updated Tableau after Key Row Normalization:")
        print(tableau)

        # Update non-key rows
        tableau = update_non_key_rows(tableau, entering_column, leaving_row)
        print("Updated Tableau after Eliminating Entering Variable from Non-Key Rows:")
        print(tableau)
    else:
        print("No leaving row found; solution is unbounded.")
else:
    print("No entering column found; optimal solution reached.")

Entering Column: 1
Leaving Row: 1
Pivot Element: 4.0
Updated Tableau after Key Row Normalization:
[[ 3.000000e+00  2.000000e+00  1.000000e+00 -1.000000e+00  0.000000e+00
   1.000000e+00  0.000000e+00  8.000000e+00]
 [ 2.500000e-01  1.000000e+00  7.500000e-01  0.000000e+00 -2.500000e-01
   0.000000e+00  2.500000e-01  3.000000e+00]
 [-3.999996e+06 -5.999995e+06 -3.999994e+06  1.000000e+06  1.000000e+06
   0.000000e+00  0.000000e+00  2.000000e+07]]
Updated Tableau after Eliminating Entering Variable from Non-Key Rows:
[[ 2.50000000e+00  0.00000000e+00 -5.00000000e-01 -1.00000000e+00
   5.00000000e-01  1.00000000e+00 -5.00000000e-01  2.00000000e+00]
 [ 2.50000000e-01  1.00000000e+00  7.50000000e-01  0.00000000e+00
  -2.50000000e-01  0.00000000e+00  2.50000000e-01  3.00000000e+00]
 [-2.49999725e+06  0.00000000e+00  5.00002250e+05  1.00000000e+06
  -4.99998750e+05  0.00000000e+00  1.49999875e+06  3.79999850e+07]]


In [ ]:
# Example usage
# Sample tableau from the previous example
tableau = np.array([
    [2.50000000e+00, 0.00000000e+00, -5.00000000e-01, -1.00000000e+00,
     5.00000000e-01, 1.00000000e+00, -5.00000000e-01, 2.00000000e+00],
    [2.50000000e-01, 1.00000000e+00, 7.50000000e-01, 0.00000000e+00,
     -2.50000000e-01, 0.00000000e+00, 2.50000000e-01, 3.00000000e+00],
    [-2.49999725e+06, 0.00000000e+00, 5.00002250e+05, 1.00000000e+06,
     -4.99998750e+05, 0.00000000e+00, 1.49999875e+06, 3.79999850e+07]
])

coeffs = [4, 5, 6, 0, 0, 1e6, 1e6]  # Coefficients from the original objective function (excluding RHS)
basis = [5, 6]  # Current basic variables

# Update the objective row
tableau = update_objective_row(tableau, coeffs, basis)
print("Updated Tableau after Recalculating the Objective Row:")
print(tableau)


Updated Tableau after Recalculating the Objective Row:
[[ 2.500000e+00  0.000000e+00 -5.000000e-01 -1.000000e+00  5.000000e-01
   1.000000e+00 -5.000000e-01  2.000000e+00]
 [ 2.500000e-01  1.000000e+00  7.500000e-01  0.000000e+00 -2.500000e-01
   0.000000e+00  2.500000e-01  3.000000e+00]
 [-2.749996e+06 -9.999950e+05 -2.499940e+05  1.000000e+06 -2.500000e+05
   0.000000e+00  1.250000e+06  5.000000e+06]]


In [ ]:
# Example usage
# Sample tableau from the previous example
tableau = np.array([
    [2.50000000e+00, 0.00000000e+00, -5.00000000e-01, -1.00000000e+00,
     5.00000000e-01, 1.00000000e+00, -5.00000000e-01, 2.00000000e+00],
    [2.50000000e-01, 1.00000000e+00, 7.50000000e-01, 0.00000000e+00,
     -2.50000000e-01, 0.00000000e+00, 2.50000000e-01, 3.00000000e+00],
    [-2.49999725e+06, 0.00000000e+00, 5.00002250e+05, 1.00000000e+06,
     -4.99998750e+05, 0.00000000e+00, 1.49999875e+06, 3.79999850e+07]
])

is_maximization = False

# Find the entering column
entering_column = identify_entering_column(tableau, is_maximization)
print(f"Entering Column: {entering_column}")

# Find the leaving row
if entering_column is not None:
    leaving_row = identify_leaving_row(tableau, entering_column)
    if leaving_row is not None:
        print(f"Leaving Row: {leaving_row}")

        # Locate the pivot element
        pivot_element = locate_pivot_element(tableau, entering_column, leaving_row)
        print(f"Pivot Element: {pivot_element}")

        # Update the key row
        tableau = update_key_row(tableau, leaving_row, pivot_element)
        print("Updated Tableau after Key Row Normalization:")
        print(tableau)

        # Update non-key rows, excluding the objective row
        tableau = update_non_key_rows(tableau, entering_column, leaving_row)
        print("Updated Tableau after Eliminating Entering Variable from Non-Key Rows:")
        print(tableau)

        # Update the objective row separately
        coeffs = [4, 5, 6, 0, 0, 1e6, 1e6]  # Coefficients from the original objective function (excluding RHS)
        basis = [5, 6]  # Current basic variables
        tableau = update_objective_row(tableau, coeffs, basis)
        print("Updated Tableau after Recalculating the Objective Row:")
        print(tableau)
    else:
        print("No leaving row found; solution is unbounded.")
else:
    print("No entering column found; optimal solution reached.")

Entering Column: 0
Leaving Row: 0
Pivot Element: 2.5
Updated Tableau after Key Row Normalization:
[[ 1.00000000e+00  0.00000000e+00 -2.00000000e-01 -4.00000000e-01
   2.00000000e-01  4.00000000e-01 -2.00000000e-01  8.00000000e-01]
 [ 2.50000000e-01  1.00000000e+00  7.50000000e-01  0.00000000e+00
  -2.50000000e-01  0.00000000e+00  2.50000000e-01  3.00000000e+00]
 [-2.49999725e+06  0.00000000e+00  5.00002250e+05  1.00000000e+06
  -4.99998750e+05  0.00000000e+00  1.49999875e+06  3.79999850e+07]]
Updated Tableau after Eliminating Entering Variable from Non-Key Rows:
[[ 1.00000000e+00  0.00000000e+00 -2.00000000e-01 -4.00000000e-01
   2.00000000e-01  4.00000000e-01 -2.00000000e-01  8.00000000e-01]
 [ 0.00000000e+00  1.00000000e+00  8.00000000e-01  1.00000000e-01
  -3.00000000e-01 -1.00000000e-01  3.00000000e-01  2.80000000e+00]
 [-2.49999725e+06  0.00000000e+00  5.00002250e+05  1.00000000e+06
  -4.99998750e+05  0.00000000e+00  1.49999875e+06  3.79999850e+07]]
Updated Tableau after Recalcula

In [ ]:
# Example usage
# Sample tableau from the previous example
tableau = np.array([
    [2.50000000e+00, 0.00000000e+00, -5.00000000e-01, -1.00000000e+00,
     5.00000000e-01, 1.00000000e+00, -5.00000000e-01, 2.00000000e+00],
    [2.50000000e-01, 1.00000000e+00, 7.50000000e-01, 0.00000000e+00,
     -2.50000000e-01, 0.00000000e+00, 2.50000000e-01, 3.00000000e+00],
    [-2.49999725e+06, 0.00000000e+00, 5.00002250e+05, 1.00000000e+06,
     -4.99998750e+05, 0.00000000e+00, 1.49999875e+06, 3.79999850e+07]
])

coeffs = [4, 5, 6, 0, 0, 1e6, 1e6]  # Coefficients from the original objective function (excluding RHS)
basis = [5, 6]  # Current basic variables
is_maximization = False
all_vars = ['x1', 'x2', 'x3', 's1', 's2', 'a1', 'a2']

# Perform one iteration of the simplex method
tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration(tableau, coeffs, basis, is_maximization, all_vars)

# Display the results after one iteration
print("Updated Tableau after One Iteration:")
print(tableau)
print("Updated Basis:", basis)
print("All Variables:", all_vars)
print("Is Optimal Solution Reached?", is_optimal)
print("Is Solution Unbounded?", is_unbounded)



Updated Tableau after One Iteration:
[[ 0.00000e+00  0.00000e+00 -2.00000e-01 -4.00000e-01  2.00000e-01
   4.00000e-01 -2.00000e-01  8.00000e-01]
 [ 0.00000e+00  1.00000e+00  8.00000e-01  1.00000e-01 -3.00000e-01
  -1.00000e-01  3.00000e-01  2.80000e+00]
 [ 0.00000e+00 -9.99995e+05 -7.99994e+05 -1.00000e+05  3.00000e+05
   1.10000e+06  7.00000e+05  2.80000e+06]]
Updated Basis: [0, 6]
All Variables: ['x1', 'x2', 'x3', 's1', 's2', 'a1', 'a2']
Is Optimal Solution Reached? False
Is Solution Unbounded? False


In [ ]:
import numpy as np

import numpy as np

# Assuming all the previously defined functions are available (including corrections)

# Initializing the problem
num_vars = 2
objective_coeffs = [1, 4]
num_constraints = 3
constraint_coeffs = [
    [1, 1],
    [1, 3],
    [1, 2]
]
constraint_types = ['>=', '>=', '<=']
b_values = [1000, 4000, 3500]
is_maximization = False

# Create the initial tableau
tableau, all_slack_vars, basis, coeffs = create_tableau_for_maximization(
    num_vars, objective_coeffs, num_constraints, constraint_coeffs, constraint_types, b_values, is_maximization
)

# Add all variable names to the list
all_vars = ['x1', 'x2'] + all_slack_vars

# Identify the artificial variables in the tableau
artificial_vars = [i for i, var in enumerate(all_vars) if 'a' in var]

# Display the initial tableau
print("\nInitial Tableau:")
print(tableau)
print("Initial Basis:", [all_vars[i] for i in basis])

# Perform iterations until the optimal solution is reached or the solution is found to be unbounded
is_optimal = False
is_unbounded = False
iteration_count = 0

# Iteration loop
while not is_optimal and not is_unbounded:
    iteration_count += 1
    print(f"\nIteration {iteration_count}:")

    # Identify the entering column
    entering_column = identify_entering_column(tableau, is_maximization)
    if entering_column is None:
        is_optimal = True
        break

    # Identify the leaving row and calculate key ratios
    key_ratios = []
    for i in range(len(basis)):
        rhs_value = tableau[i, -1]
        entering_value = tableau[i, entering_column]
        if entering_value > 0:
            key_ratios.append(rhs_value / entering_value)
        else:
            key_ratios.append(np.inf)

    print(f"Key Ratios for Leaving Variable Selection: {key_ratios}")

    # Find the leaving row
    leaving_row = identify_leaving_row(tableau, entering_column)
    if leaving_row is None:
        is_unbounded = True
        break

    # Perform a simplex iteration
    tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration(
        tableau, coeffs, basis, is_maximization, all_vars, artificial_vars
    )

    # Display updated tableau
    print("Updated Tableau:")
    print(tableau)
    print("Updated Basis:", [all_vars[i] for i in basis])

# Display the final results
print("\nFinal Tableau:")
print(tableau)
print("Final Basis:", [all_vars[i] for i in basis])
print("All Variables:", all_vars)
print("Is Optimal Solution Reached?", is_optimal)
print("Is Solution Unbounded?", is_unbounded)

# Extract and display the optimal solution
if is_optimal:
    optimal_values, objective_value = extract_optimal_solution(tableau, basis, all_vars)
    print("\nOptimal Solution:")
    for var, value in optimal_values.items():
        print(f"{var} = {value}")
    print(f"Optimal Value of the Objective Function: {objective_value}")



Initial Basis Indices: [5, 6, 4]
Initial Basis Variables and Their Values:
Variable a2 (Index 5): Value = M
Variable s3 (Index 6): Value = 4000.0
Variable s2 (Index 4): Value = 3500.0

Initial Tableau:
[[ 1.000000e+00  1.000000e+00 -1.000000e+00  0.000000e+00  0.000000e+00
   1.000000e+00  0.000000e+00  1.000000e+03]
 [ 1.000000e+00  3.000000e+00  0.000000e+00 -1.000000e+00  0.000000e+00
   0.000000e+00  1.000000e+00  4.000000e+03]
 [ 1.000000e+00  2.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00
   0.000000e+00  0.000000e+00  3.500000e+03]
 [-1.999999e+06 -3.999996e+06  1.000000e+06  1.000000e+06  0.000000e+00
   0.000000e+00  0.000000e+00  5.000000e+09]]
Initial Basis: ['a2', 's3', 's2']

Iteration 1:
Key Ratios for Leaving Variable Selection: [1000.0, 1333.3333333333333, 1750.0]
Updated Tableau:
[[ 1.000000e+00  1.000000e+00 -1.000000e+00  0.000000e+00  0.000000e+00
   1.000000e+00  0.000000e+00  1.000000e+03]
 [-2.000000e+00  0.000000e+00  3.000000e+00 -1.000000e+00  0.000000

In [ ]:
# Example usage with the changes included.
# Initializing the problem
num_vars = 2
objective_coeffs = [1, 4]
num_constraints = 3
constraint_coeffs = [
    [1, 1],
    [1, 3],
    [1, 2]
]
constraint_types = ['>=', '>=', '<=']
b_values = [1000, 4000, 3500]
is_maximization = False

# Create the initial tableau
tableau, all_slack_vars, basis, coeffs = create_tableau_for_maximization(
    num_vars, objective_coeffs, num_constraints, constraint_coeffs, constraint_types, b_values, is_maximization
)

# Add all variable names to the list
all_vars = ['x1', 'x2'] + all_slack_vars

# Display the initial tableau
print("\nInitial Tableau:")
print(tableau)

Initial Basis Indices: [5, 6, 4]
Initial Basis Variables and Their Values:
Variable a-1 (Index 5): Value = M
Variable a0 (Index 6): Value = M
Variable s2 (Index 4): Value = 0

Initial Tableau:
[[ 1.000000e+00  1.000000e+00 -1.000000e+00  0.000000e+00  0.000000e+00
   1.000000e+00  0.000000e+00  1.000000e+03]
 [ 1.000000e+00  3.000000e+00  0.000000e+00 -1.000000e+00  0.000000e+00
   0.000000e+00  1.000000e+00  4.000000e+03]
 [ 1.000000e+00  2.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00
   0.000000e+00  0.000000e+00  3.500000e+03]
 [-1.999999e+06 -3.999996e+06  1.000000e+06  1.000000e+06  0.000000e+00
   0.000000e+00  0.000000e+00  5.000000e+09]]


In [ ]:
import numpy as np

# Assuming all the previously defined functions are available (including corrections)

# Define the problem
num_vars = 2
objective_coeffs = [1, 2]
num_constraints = 2
constraint_coeffs = [[1, 1], [1, 2]]
constraint_types = ['>=', '>=']
b_values = [8, 15]
is_maximization = False

# Create the initial tableau
tableau, all_slack_vars, basis, coeffs = create_tableau_for_maximization(
    num_vars, objective_coeffs, num_constraints, constraint_coeffs, constraint_types, b_values, is_maximization
)

# Add all variable names to the list
all_vars = ['x1', 'x2'] + all_slack_vars

# Identify the artificial variables in the tableau
artificial_vars = [i for i, var in enumerate(all_vars) if 'a' in var]

# Display the initial tableau
print("\nInitial Tableau:")
print(tableau)
print("Initial Basis Indices:", basis)
print("Initial Basis Variables and Their Values:")

# Correct the basis variable values retrieval
for row_idx, var_idx in enumerate(basis):
    var_name = all_vars[var_idx]
    rhs_value = tableau[row_idx, -1]
    value = 'M' if var_idx in artificial_vars else rhs_value
    print(f"Variable {var_name} (Index {var_idx}): Value = {value}")

# Perform iterations until the optimal solution is reached or the solution is found to be unbounded
is_optimal = False
is_unbounded = False
iteration_count = 0

# Iteration loop
while not is_optimal and not is_unbounded:
    iteration_count += 1
    print(f"\nIteration {iteration_count}:")

    # Identify the entering column
    entering_column = identify_entering_column(tableau, is_maximization)
    if entering_column is None:
        is_optimal = True
        break

    # Identify the leaving row and calculate key ratios
    key_ratios = []
    for i in range(len(basis)):
        rhs_value = tableau[i, -1]
        entering_value = tableau[i, entering_column]
        if entering_value > 0:
            key_ratios.append(rhs_value / entering_value)
        else:
            key_ratios.append(np.inf)

    print(f"Key Ratios for Leaving Variable Selection: {key_ratios}")

    # Find the leaving row
    leaving_row = identify_leaving_row(tableau, entering_column)
    if leaving_row is None:
        is_unbounded = True
        break

    # Perform a simplex iteration
    tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration(
        tableau, coeffs, basis, is_maximization, all_vars, artificial_vars
    )

    # Display updated tableau
    print("Updated Tableau:")
    print(tableau)
    print("Updated Basis:", [all_vars[i] for i in basis])

# Display the final results
print("\nFinal Tableau:")
print(tableau)
print("Final Basis:", [all_vars[i] for i in basis])
print("All Variables:", all_vars)
print("Is Optimal Solution Reached?", is_optimal)
print("Is Solution Unbounded?", is_unbounded)

# Extract and display the optimal solution
if is_optimal:
    optimal_values, objective_value = extract_optimal_solution(tableau, basis, all_vars)
    print("\nOptimal Solution:")
    for var, value in optimal_values.items():
        print(f"{var} = {value}")
    print(f"Optimal Value of the Objective Function: {objective_value}")



Initial Basis Indices: [4, 5]
Initial Basis Variables and Their Values:
Variable s2 (Index 4): Value = 8.0
Variable a2 (Index 5): Value = M

Initial Tableau:
[[ 1.000000e+00  1.000000e+00 -1.000000e+00  0.000000e+00  1.000000e+00
   0.000000e+00  8.000000e+00]
 [ 1.000000e+00  2.000000e+00  0.000000e+00 -1.000000e+00  0.000000e+00
   1.000000e+00  1.500000e+01]
 [-1.999999e+06 -2.999998e+06  1.000000e+06  1.000000e+06  0.000000e+00
   0.000000e+00  2.300000e+07]]
Initial Basis Indices: [4, 5]
Initial Basis Variables and Their Values:
Variable s2 (Index 4): Value = 8.0
Variable a2 (Index 5): Value = M

Iteration 1:
Key Ratios for Leaving Variable Selection: [8.0, 7.5]
Updated Tableau:
[[ 5.000000e-01  0.000000e+00 -1.000000e+00  5.000000e-01  1.000000e+00
  -5.000000e-01  5.000000e-01]
 [ 5.000000e-01  1.000000e+00  0.000000e+00 -5.000000e-01  0.000000e+00
   5.000000e-01  7.500000e+00]
 [-5.000000e+05  0.000000e+00  1.000000e+06 -4.999990e+05  0.000000e+00
   1.499999e+06  5.000150e+05

In [ ]:
import numpy as np

# Assuming all the previously defined functions are available (including corrections)

# Define the problem
num_vars = 2
objective_coeffs = [12, 16]
num_constraints = 2
constraint_coeffs = [[1, 2], [1, 1]]
constraint_types = ['>=', '>=']
b_values = [40, 30]
is_maximization = False

# Create the initial tableau
tableau, all_slack_vars, basis, coeffs = create_tableau_for_maximization(
    num_vars, objective_coeffs, num_constraints, constraint_coeffs, constraint_types, b_values, is_maximization
)

# Add all variable names to the list
all_vars = ['x1', 'x2'] + all_slack_vars

# Identify the artificial variables in the tableau
artificial_vars = [i for i, var in enumerate(all_vars) if 'a' in var]

# Display the initial tableau
print("\nInitial Tableau:")
print(tableau)
print("Initial Basis Indices:", basis)
print("Initial Basis Variables and Their Values:")

# Correct the basis variable values retrieval
for row_idx, var_idx in enumerate(basis):
    var_name = all_vars[var_idx]
    rhs_value = tableau[row_idx, -1]
    value = 'M' if var_idx in artificial_vars else rhs_value
    print(f"Variable {var_name} (Index {var_idx}): Value = {value}")

# Perform iterations until the optimal solution is reached or the solution is found to be unbounded
is_optimal = False
is_unbounded = False
iteration_count = 0

# Iteration loop
while not is_optimal and not is_unbounded:
    iteration_count += 1
    print(f"\nIteration {iteration_count}:")

    # Identify the entering column
    entering_column = identify_entering_column(tableau, is_maximization)
    if entering_column is None:
        is_optimal = True
        break

    # Identify the leaving row and calculate key ratios
    key_ratios = []
    for i in range(len(basis)):
        rhs_value = tableau[i, -1]
        entering_value = tableau[i, entering_column]
        if entering_value > 0:
            key_ratios.append(rhs_value / entering_value)
        else:
            key_ratios.append(np.inf)

    print(f"Key Ratios for Leaving Variable Selection: {key_ratios}")

    # Find the leaving row
    leaving_row = identify_leaving_row(tableau, entering_column)
    if leaving_row is None:
        is_unbounded = True
        break

    # Perform a simplex iteration
    tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration(
        tableau, coeffs, basis, is_maximization, all_vars, artificial_vars
    )

    # Display updated tableau
    print("Updated Tableau:")
    print(tableau)
    print("Updated Basis:", [all_vars[i] for i in basis])

# Display the final results
print("\nFinal Tableau:")
print(tableau)
print("Final Basis:", [all_vars[i] for i in basis])
print("All Variables:", all_vars)
print("Is Optimal Solution Reached?", is_optimal)
print("Is Solution Unbounded?", is_unbounded)

# Extract and display the optimal solution
if is_optimal:
    optimal_values, objective_value = extract_optimal_solution(tableau, basis, all_vars)
    print("\nOptimal Solution:")
    for var, value in optimal_values.items():
        print(f"{var} = {value}")
    print(f"Optimal Value of the Objective Function: {objective_value}")










Initial Basis Indices: [4, 5]
Initial Basis Variables and Their Values:
Variable s2 (Index 4): Value = 40.0
Variable a2 (Index 5): Value = M

Initial Tableau:
[[ 1.000000e+00  2.000000e+00 -1.000000e+00  0.000000e+00  1.000000e+00
   0.000000e+00  4.000000e+01]
 [ 1.000000e+00  1.000000e+00  0.000000e+00 -1.000000e+00  0.000000e+00
   1.000000e+00  3.000000e+01]
 [-1.999988e+06 -2.999984e+06  1.000000e+06  1.000000e+06  0.000000e+00
   0.000000e+00  7.000000e+07]]
Initial Basis Indices: [4, 5]
Initial Basis Variables and Their Values:
Variable s2 (Index 4): Value = 40.0
Variable a2 (Index 5): Value = M

Iteration 1:
Key Ratios for Leaving Variable Selection: [20.0, 30.0]
Updated Tableau:
[[ 5.000000e-01  1.000000e+00 -5.000000e-01  0.000000e+00  5.000000e-01
   0.000000e+00  2.000000e+01]
 [ 5.000000e-01  0.000000e+00  5.000000e-01 -1.000000e+00 -5.000000e-01
   1.000000e+00  1.000000e+01]
 [-4.999960e+05  0.000000e+00 -4.999920e+05  1.000000e+06  1.499992e+06
   0.000000e+00  1.000032

In [ ]:
import numpy as np

# Assuming all the previously defined functions are available (including corrections)

# Define the problem
num_vars = 2
objective_coeffs = [2000, 1500]
num_constraints = 3
constraint_coeffs = [[6, 2], [2, 4], [4, 12]]
constraint_types = ['>=', '>=', '>=']
b_values = [8, 12, 24]
is_maximization = False

# Create the initial tableau
tableau, all_slack_vars, basis, coeffs = create_tableau_for_maximization(
    num_vars, objective_coeffs, num_constraints, constraint_coeffs, constraint_types, b_values, is_maximization
)

# Add all variable names to the list
all_vars = ['x1', 'x2'] + all_slack_vars

# Identify the artificial variables in the tableau
artificial_vars = [i for i, var in enumerate(all_vars) if 'a' in var]

# Display the initial tableau
print("\nInitial Tableau:")
print(tableau)
print("Initial Basis Indices:", basis)
print("Initial Basis Variables and Their Values:")

# Correct the basis variable values retrieval
for row_idx, var_idx in enumerate(basis):
    var_name = all_vars[var_idx]
    rhs_value = tableau[row_idx, -1]
    value = 'M' if var_idx in artificial_vars else rhs_value
    print(f"Variable {var_name} (Index {var_idx}): Value = {value}")

# Perform iterations until the optimal solution is reached or the solution is found to be unbounded
is_optimal = False
is_unbounded = False
iteration_count = 0

# Iteration loop
while not is_optimal and not is_unbounded:
    iteration_count += 1
    print(f"\nIteration {iteration_count}:")

    # Identify the entering column
    entering_column = identify_entering_column(tableau, is_maximization)
    if entering_column is None:
        is_optimal = True
        break

    # Identify the leaving row and calculate key ratios
    key_ratios = []
    for i in range(len(basis)):
        rhs_value = tableau[i, -1]
        entering_value = tableau[i, entering_column]
        if entering_value > 0:
            key_ratios.append(rhs_value / entering_value)
        else:
            key_ratios.append(np.inf)

    print(f"Key Ratios for Leaving Variable Selection: {key_ratios}")

    # Find the leaving row
    leaving_row = identify_leaving_row(tableau, entering_column)
    if leaving_row is None:
        is_unbounded = True
        break

    # Perform a simplex iteration
    tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration(
        tableau, coeffs, basis, is_maximization, all_vars, artificial_vars
    )

    # Display updated tableau
    print("Updated Tableau:")
    print(tableau)
    print("Updated Basis:", [all_vars[i] for i in basis])

# Display the final results
print("\nFinal Tableau:")
print(tableau)
print("Final Basis:", [all_vars[i] for i in basis])
print("All Variables:", all_vars)
print("Is Optimal Solution Reached?", is_optimal)
print("Is Solution Unbounded?", is_unbounded)

# Extract and display the optimal solution
if is_optimal:
    optimal_values, objective_value = extract_optimal_solution(tableau, basis, all_vars)
    print("\nOptimal Solution:")
    for var, value in optimal_values.items():
        print(f"{var} = {value}")
    print(f"Optimal Value of the Objective Function: {objective_value}")

Initial Basis Indices: [5, 6, 7]
Initial Basis Variables and Their Values:
Variable a-1 (Index 5): Value = M
Variable a0 (Index 6): Value = M
Variable a1 (Index 7): Value = M

Initial Tableau:
[[ 6.00000e+00  2.00000e+00 -1.00000e+00  0.00000e+00  0.00000e+00
   1.00000e+00  0.00000e+00  0.00000e+00  8.00000e+00]
 [ 2.00000e+00  4.00000e+00  0.00000e+00 -1.00000e+00  0.00000e+00
   0.00000e+00  1.00000e+00  0.00000e+00  1.20000e+01]
 [ 4.00000e+00  1.20000e+01  0.00000e+00  0.00000e+00 -1.00000e+00
   0.00000e+00  0.00000e+00  1.00000e+00  2.40000e+01]
 [-1.19980e+07 -1.79985e+07  1.00000e+06  1.00000e+06  1.00000e+06
   0.00000e+00  0.00000e+00  0.00000e+00  4.40000e+07]]
Initial Basis Indices: [5, 6, 7]
Initial Basis Variables and Their Values:
Variable a2 (Index 5): Value = M
Variable s3 (Index 6): Value = 12.0
Variable a3 (Index 7): Value = M

Iteration 1:
Key Ratios for Leaving Variable Selection: [4.0, 3.0, 2.0]
Updated Tableau:
[[ 5.33333333e+00  0.00000000e+00 -1.00000000e+00  

In [ ]:
import numpy as np

# Assuming all the previously defined functions are available (including corrections)

# Define the problem
num_vars = 2
objective_coeffs = [3, 2]
num_constraints = 3
constraint_coeffs = [[1, 1], [2, 1], [0, 1]]
constraint_types = ['<=', '>=', '=']
b_values = [4, 5, 2]
is_maximization = False

# Create the initial tableau
tableau, all_slack_vars, basis, coeffs = create_tableau_for_maximization(
    num_vars, objective_coeffs, num_constraints, constraint_coeffs, constraint_types, b_values, is_maximization
)

# Add all variable names to the list
all_vars = ['x1', 'x2'] + all_slack_vars

# Identify the artificial variables in the tableau
artificial_vars = [i for i, var in enumerate(all_vars) if 'a' in var]

# Display the initial tableau
print("\nInitial Tableau:")
print(tableau)
print("Initial Basis Indices:", basis)
print("Initial Basis Variables and Their Values:")

# Correct the basis variable values retrieval
for row_idx, var_idx in enumerate(basis):
    var_name = all_vars[var_idx]
    rhs_value = tableau[row_idx, -1]
    value = 'M' if var_idx in artificial_vars else rhs_value
    print(f"Variable {var_name} (Index {var_idx}): Value = {value}")

# Perform iterations until the optimal solution is reached or the solution is found to be unbounded
is_optimal = False
is_unbounded = False
iteration_count = 0

# Iteration loop
while not is_optimal and not is_unbounded:
    iteration_count += 1
    print(f"\nIteration {iteration_count}:")

    # Identify the entering column
    entering_column = identify_entering_column(tableau, is_maximization)
    if entering_column is None:
        is_optimal = True
        break

    # Identify the leaving row and calculate key ratios
    key_ratios = []
    for i in range(len(basis)):
        rhs_value = tableau[i, -1]
        entering_value = tableau[i, entering_column]
        if entering_value > 0:
            key_ratios.append(rhs_value / entering_value)
        else:
            key_ratios.append(np.inf)

    print(f"Key Ratios for Leaving Variable Selection: {key_ratios}")

    # Find the leaving row
    leaving_row = identify_leaving_row(tableau, entering_column)
    if leaving_row is None:
        is_unbounded = True
        break

    # Perform a simplex iteration
    tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration(
        tableau, coeffs, basis, is_maximization, all_vars, artificial_vars
    )

    # Display updated tableau
    print("Updated Tableau:")
    print(tableau)
    print("Updated Basis:", [all_vars[i] for i in basis])

# Display the final results
print("\nFinal Tableau:")
print(tableau)
print("Final Basis:", [all_vars[i] for i in basis])
print("All Variables:", all_vars)
print("Is Optimal Solution Reached?", is_optimal)
print("Is Solution Unbounded?", is_unbounded)

# Extract and display the optimal solution
if is_optimal:
    optimal_values, objective_value = extract_optimal_solution(tableau, basis, all_vars)
    print("\nOptimal Solution:")
    for var, value in optimal_values.items():
        print(f"{var} = {value}")
    print(f"Optimal Value of the Objective Function: {objective_value}")

Initial Basis Indices: [2, 4, 5]
Initial Basis Variables and Their Values:
Variable s1 (Index 2): Value = 0
Variable a0 (Index 4): Value = M
Variable a1 (Index 5): Value = M

Initial Tableau:
[[ 1.000000e+00  1.000000e+00  1.000000e+00  0.000000e+00  0.000000e+00
   0.000000e+00  4.000000e+00]
 [ 2.000000e+00  1.000000e+00  0.000000e+00 -1.000000e+00  1.000000e+00
   0.000000e+00  5.000000e+00]
 [ 0.000000e+00  1.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
   1.000000e+00  2.000000e+00]
 [-1.999997e+06 -1.999998e+06  0.000000e+00  1.000000e+06  0.000000e+00
   0.000000e+00  7.000000e+06]]
Initial Basis Indices: [2, 4, 5]
Initial Basis Variables and Their Values:
Variable s1 (Index 2): Value = 4.0
Variable a1 (Index 4): Value = M
Variable a2 (Index 5): Value = M

Iteration 1:
Key Ratios for Leaving Variable Selection: [4.0, 5.0, 2.0]
Updated Tableau:
[[ 1.000000e+00  0.000000e+00  1.000000e+00  0.000000e+00  0.000000e+00
  -1.000000e+00  2.000000e+00]
 [ 2.000000e+00  0.000000e

In [ ]:
# Example usage
num_vars = 3
objective_coeffs = [3, 5, 4]  # Coefficients for the objective function (maximize)
num_constraints = 3
constraint_coeffs = [
    [2, 3, 0],
    [0, 2, 5],
    [3, 2, 4]
]
constraint_types = ['<=', '<=', '<=']
b_values = [8, 10, 15]
is_maximization = True  # Set to True for a maximization problem

# Create the initial tableau
tableau, all_slack_vars, basis, coeffs = create_tableau_for_maximization(
    num_vars, objective_coeffs, num_constraints, constraint_coeffs, constraint_types, b_values, is_maximization
)

# Perform the simplex method
optimal_solution, optimal_value = simplex_method_leq(tableau, objective_coeffs)

Initial Basis Indices: [3, 4, 5]
Initial Basis Variables and Their Values:
Variable s1 (Index 3): Value = 8.0
Variable s2 (Index 4): Value = 10.0
Variable s3 (Index 5): Value = 15.0

Iteration 0:
--> Entering column: 1
--> Leaving row: 0
--> Pivot element: 3.0

Iteration 1:
--> Entering column: 2
--> Leaving row: 1
--> Pivot element: 5.0

Iteration 2:
--> Entering column: 0
--> Leaving row: 2
--> Pivot element: 2.7333333333333334

Iteration 3:
Optimal solution found!

Optimal solution:
x1 = 2.1707317073170733
x2 = 1.219512195121951
x3 = 1.5121951219512195
Optimal value: 18.65853658536585


In [ ]:
import numpy as np

# Assuming all the previously defined functions are available (including corrections)

# Define the problem
num_vars = 2
objective_coeffs = [1, 1]
num_constraints = 2
constraint_coeffs = [[2, 4], [1, 7]]
constraint_types = ['>=', '>=']
b_values = [4, 7]
is_maximization = False

# Create the initial tableau
tableau, all_slack_vars, basis, coeffs = create_tableau_for_maximization(
    num_vars, objective_coeffs, num_constraints, constraint_coeffs, constraint_types, b_values, is_maximization
)

# Add all variable names to the list
all_vars = ['x1', 'x2'] + all_slack_vars

# Identify the artificial variables in the tableau
artificial_vars = [i for i, var in enumerate(all_vars) if 'a' in var]

# Display the initial tableau
print("\nInitial Tableau:")
print(tableau)
print("Initial Basis Indices:", basis)
print("Initial Basis Variables and Their Values:")

# Correct the basis variable values retrieval
for row_idx, var_idx in enumerate(basis):
    var_name = all_vars[var_idx]
    rhs_value = tableau[row_idx, -1]
    value = 'M' if var_idx in artificial_vars else rhs_value
    print(f"Variable {var_name} (Index {var_idx}): Value = {value}")

# Perform iterations until the optimal solution is reached or the solution is found to be unbounded
is_optimal = False
is_unbounded = False
iteration_count = 0

# Iteration loop
while not is_optimal and not is_unbounded:
    iteration_count += 1
    print(f"\nIteration {iteration_count}:")

    # Identify the entering column
    entering_column = identify_entering_column(tableau, is_maximization)
    if entering_column is None:
        is_optimal = True
        break

    # Identify the leaving row and calculate key ratios
    key_ratios = []
    for i in range(len(basis)):
        rhs_value = tableau[i, -1]
        entering_value = tableau[i, entering_column]
        if entering_value > 0:
            key_ratios.append(rhs_value / entering_value)
        else:
            key_ratios.append(np.inf)

    print(f"Key Ratios for Leaving Variable Selection: {key_ratios}")

    # Find the leaving row
    leaving_row = identify_leaving_row(tableau, entering_column)
    if leaving_row is None:
        is_unbounded = True
        break

    # Perform a simplex iteration
    tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration(
        tableau, coeffs, basis, is_maximization, all_vars, artificial_vars
    )

    # Display updated tableau
    print("Updated Tableau:")
    print(tableau)
    print("Updated Basis:", [all_vars[i] for i in basis])

# Display the final results
print("\nFinal Tableau:")
print(tableau)
print("Final Basis:", [all_vars[i] for i in basis])
print("All Variables:", all_vars)
print("Is Optimal Solution Reached?", is_optimal)
print("Is Solution Unbounded?", is_unbounded)

# Extract and display the optimal solution
if is_optimal:
    optimal_values, objective_value = extract_optimal_solution(tableau, basis, all_vars)
    print("\nOptimal Solution:")
    for var, value in optimal_values.items():
        print(f"{var} = {value}")
    print(f"Optimal Value of the Objective Function: {objective_value}")

Initial Basis Indices: [4, 5]
Initial Basis Variables and Their Values:
Variable s2 (Index 4): Value = 4.0
Variable a2 (Index 5): Value = M

Initial Tableau:
[[ 2.0000000e+00  4.0000000e+00 -1.0000000e+00  0.0000000e+00
   1.0000000e+00  0.0000000e+00  4.0000000e+00]
 [ 1.0000000e+00  7.0000000e+00  0.0000000e+00 -1.0000000e+00
   0.0000000e+00  1.0000000e+00  7.0000000e+00]
 [-2.9999990e+06 -1.0999999e+07  1.0000000e+06  1.0000000e+06
   0.0000000e+00  0.0000000e+00  1.1000000e+07]]
Initial Basis Indices: [4, 5]
Initial Basis Variables and Their Values:
Variable s2 (Index 4): Value = 4.0
Variable a2 (Index 5): Value = M

Iteration 1:
Key Ratios for Leaving Variable Selection: [1.0, 1.0]
Updated Tableau:
[[ 5.00000000e-01  1.00000000e+00 -2.50000000e-01  0.00000000e+00
   2.50000000e-01  0.00000000e+00  1.00000000e+00]
 [-2.50000000e+00  0.00000000e+00  1.75000000e+00 -1.00000000e+00
  -1.75000000e+00  1.00000000e+00  0.00000000e+00]
 [ 2.50000050e+06  0.00000000e+00 -1.74999975e+06  1

In [ ]:
import numpy as np

# Assuming all the previously defined functions are available (including corrections)

# Define the problem
num_vars = 2
objective_coeffs = [5, 3]
num_constraints = 3
constraint_coeffs = [[2, 4], [2, 2], [5,2]]
constraint_types = ['<=', '=', '>=']
b_values = [12, 10, 10]
is_maximization = False

# Create the initial tableau
tableau, all_slack_vars, basis, coeffs = create_tableau_for_maximization(
    num_vars, objective_coeffs, num_constraints, constraint_coeffs, constraint_types, b_values, is_maximization
)

# Add all variable names to the list
all_vars = ['x1', 'x2'] + all_slack_vars

# Identify the artificial variables in the tableau
artificial_vars = [i for i, var in enumerate(all_vars) if 'a' in var]

# Display the initial tableau
print("\nInitial Tableau:")
print(tableau)
print("Initial Basis Indices:", basis)
print("Initial Basis Variables and Their Values:")

# Correct the basis variable values retrieval
for row_idx, var_idx in enumerate(basis):
    var_name = all_vars[var_idx]
    rhs_value = tableau[row_idx, -1]
    value = 'M' if var_idx in artificial_vars else rhs_value
    print(f"Variable {var_name} (Index {var_idx}): Value = {value}")

# Perform iterations until the optimal solution is reached or the solution is found to be unbounded
is_optimal = False
is_unbounded = False
iteration_count = 0

# Iteration loop
while not is_optimal and not is_unbounded:
    iteration_count += 1
    print(f"\nIteration {iteration_count}:")

    # Identify the entering column
    entering_column = identify_entering_column(tableau, is_maximization)
    if entering_column is None:
        is_optimal = True
        break

    # Identify the leaving row and calculate key ratios
    key_ratios = []
    for i in range(len(basis)):
        rhs_value = tableau[i, -1]
        entering_value = tableau[i, entering_column]
        if entering_value > 0:
            key_ratios.append(rhs_value / entering_value)
        else:
            key_ratios.append(np.inf)

    print(f"Key Ratios for Leaving Variable Selection: {key_ratios}")

    # Find the leaving row
    leaving_row = identify_leaving_row(tableau, entering_column)
    if leaving_row is None:
        is_unbounded = True
        break

    # Perform a simplex iteration
    tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration(
        tableau, coeffs, basis, is_maximization, all_vars, artificial_vars
    )

    # Display updated tableau
    print("Updated Tableau:")
    print(tableau)
    print("Updated Basis:", [all_vars[i] for i in basis])

# Display the final results
print("\nFinal Tableau:")
print(tableau)
print("Final Basis:", [all_vars[i] for i in basis])
print("All Variables:", all_vars)
print("Is Optimal Solution Reached?", is_optimal)
print("Is Solution Unbounded?", is_unbounded)

# Extract and display the optimal solution
if is_optimal:
    optimal_values, objective_value = extract_optimal_solution(tableau, basis, all_vars)
    print("\nOptimal Solution:")
    for var, value in optimal_values.items():
        print(f"{var} = {value}")
    print(f"Optimal Value of the Objective Function: {objective_value}")

Initial Basis Indices: [2, 4, 5]
Initial Basis Variables and Their Values:
Variable s1 (Index 2): Value = 0
Variable a0 (Index 4): Value = M
Variable a1 (Index 5): Value = M

Initial Tableau:
[[ 2.000000e+00  4.000000e+00  1.000000e+00  0.000000e+00  0.000000e+00
   0.000000e+00  1.200000e+01]
 [ 2.000000e+00  2.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00
   0.000000e+00  1.000000e+01]
 [ 5.000000e+00  2.000000e+00  0.000000e+00 -1.000000e+00  0.000000e+00
   1.000000e+00  1.000000e+01]
 [-6.999995e+06 -3.999997e+06  0.000000e+00  1.000000e+06  0.000000e+00
   0.000000e+00  2.000000e+07]]
Initial Basis Indices: [2, 4, 5]
Initial Basis Variables and Their Values:
Variable s1 (Index 2): Value = 12.0
Variable s3 (Index 4): Value = 10.0
Variable a2 (Index 5): Value = M

Iteration 1:
Key Ratios for Leaving Variable Selection: [6.0, 5.0, 2.0]
Updated Tableau:
[[ 0.000000e+00  3.200000e+00  1.000000e+00  4.000000e-01  0.000000e+00
  -4.000000e-01  8.000000e+00]
 [ 0.000000e+00  1.200

In [ ]:
# Define the hydropower plant optimization problem
num_vars = 3
objective_coeffs = [3000, 2500, 4000]  # Example coefficients for the objective function (maximize)
num_constraints = 3
constraint_coeffs = [
    [6, 2, 4],    # Water balance constraint
    [2, 4, 3],    # Turbine capacity constraint
    [5, 2, 1]     # Environmental release constraint
]
constraint_types = ['<=', '<=', '<=']
b_values = [50, 40, 30]
is_maximization = True  # Set to True for a maximization problem

# Create the initial tableau
tableau, all_slack_vars, basis, coeffs = create_tableau_for_maximization(
    num_vars, objective_coeffs, num_constraints, constraint_coeffs, constraint_types, b_values, is_maximization
)

# Print the initial tableau and basis details
print("\nInitial Basis Indices:", basis)
print("Initial Basis Variables and Their Values:")
for i in basis:
    var_name = f'a{i - num_vars + 1}' if i >= num_vars + len(all_slack_vars) else f'x{i + 1}' if i < num_vars else all_slack_vars[i - num_vars]
    value = 'M' if i >= num_vars + len(all_slack_vars) else tableau[i - len(basis), -1]
    print(f"Variable {var_name} (Index {i}): Value = {value}")

print("\nInitial Tableau:")
print(tableau)

# Perform the Simplex method
optimal_solution, optimal_value = simplex_method_leq(tableau, objective_coeffs)

Initial Basis Indices: [3, 4, 5]
Initial Basis Variables and Their Values:
Variable s1 (Index 3): Value = 0
Variable s2 (Index 4): Value = 0
Variable s3 (Index 5): Value = 0

Initial Basis Indices: [3, 4, 5]
Initial Basis Variables and Their Values:
Variable s1 (Index 3): Value = 50.0
Variable s2 (Index 4): Value = 40.0
Variable s3 (Index 5): Value = 30.0

Initial Tableau:
[[ 6.0e+00  2.0e+00  4.0e+00  1.0e+00  0.0e+00  0.0e+00  5.0e+01]
 [ 2.0e+00  4.0e+00  3.0e+00  0.0e+00  1.0e+00  0.0e+00  4.0e+01]
 [ 5.0e+00  2.0e+00  1.0e+00  0.0e+00  0.0e+00  1.0e+00  3.0e+01]
 [-3.0e+03 -2.5e+03 -4.0e+03  0.0e+00  0.0e+00  0.0e+00  0.0e+00]]

Iteration 0:
--> Entering column: 2
--> Leaving row: 0
--> Pivot element: 4.0

Iteration 1:
--> Entering column: 1
--> Leaving row: 1
--> Pivot element: 2.5

Iteration 2:
Optimal solution found!

Optimal solution:
x1 = 0.0
x2 = 1.0
x3 = 12.0
Optimal value: 50500.0


In [ ]:
# Define the problem
num_vars = 3
objective_coeffs = [5000, 4000, 6000]  # Coefficients for the objective function (maximize)
num_constraints = 3
constraint_coeffs = [
    [3, 2, 1],  # Coefficients for constraint 1
    [1, 4, 3],  # Coefficients for constraint 2
    [1, 2, 2]   # Coefficients for constraint 3 (>= type)
]
constraint_types = ['<=', '<=', '>=']  # Types of constraints
b_values = [1200, 1500, 800]  # Right-hand side values
is_maximization = True  # This is a maximization problem

# Create the initial tableau
tableau, all_slack_vars, basis, coeffs = create_tableau_for_maximization(
    num_vars, objective_coeffs, num_constraints, constraint_coeffs, constraint_types, b_values, is_maximization
)

# Add all variable names to the list
all_vars = ['x1', 'x2', 'x3'] + all_slack_vars

# Identify the artificial variables in the tableau
artificial_vars = [i for i, var in enumerate(all_vars) if 'a' in var]

# Perform iterations using simplex_iteration
is_optimal = False
is_unbounded = False
iteration_count = 0

while not is_optimal and not is_unbounded:
    iteration_count += 1
    print(f"\nIteration {iteration_count}:")
    tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration(
        tableau, coeffs, basis, is_maximization, all_vars, artificial_vars
    )
    print("Updated Tableau:")
    print(tableau)
    print("Updated Basis:", [all_vars[i] for i in basis])

# Extract and display the optimal solution
if is_optimal:
    optimal_values, objective_value = extract_optimal_solution(tableau, basis, all_vars)
    print("\nOptimal Solution:")
    for var, value in optimal_values.items():
        print(f"{var} = {value}")
    print(f"Optimal Value of the Objective Function: {objective_value}")

elif is_unbounded:
    print("The solution is unbounded.")

Initial Basis Indices: [3, 4, 6]
Initial Basis Variables and Their Values:
Variable s1 (Index 3): Value = 1200.0
Variable s2 (Index 4): Value = 1500.0
Variable a1 (Index 6): Value = -M

Iteration 1:
Updated Tableau:
[[ 2.50e+00  0.00e+00 -5.00e-01  1.00e+00 -5.00e-01  0.00e+00  0.00e+00
   4.50e+02]
 [ 2.50e-01  1.00e+00  7.50e-01  0.00e+00  2.50e-01  0.00e+00  0.00e+00
   3.75e+02]
 [ 5.00e-01  0.00e+00  5.00e-01  0.00e+00 -5.00e-01 -1.00e+00  1.00e+00
   5.00e+01]
 [ 4.96e+05  0.00e+00  4.97e+05  0.00e+00 -4.99e+05 -1.00e+06  0.00e+00
  -5.15e+07]]
Updated Basis: ['s1', 'x2', 'a1']

Iteration 2:
Updated Tableau:
[[ 3.00e+00  0.00e+00  0.00e+00  1.00e+00 -1.00e+00 -1.00e+00  1.00e+00
   5.00e+02]
 [-5.00e-01  1.00e+00  0.00e+00  0.00e+00  1.00e+00  1.50e+00 -1.50e+00
   3.00e+02]
 [ 1.00e+00  0.00e+00  1.00e+00  0.00e+00 -1.00e+00 -2.00e+00  2.00e+00
   1.00e+02]
 [-1.00e+03  0.00e+00  0.00e+00  0.00e+00 -2.00e+03 -6.00e+03 -9.94e+05
  -1.80e+06]]
Updated Basis: ['s1', 'x2', 'x3']

Op

In [ ]:
from pulp import LpMaximize, LpMinimize, LpProblem, LpVariable
import pulp

# Custom Simplex Solver Function Definitions here...

# Define the problem
num_vars = 3
objective_coeffs = [5000, 4000, 6000]  # Coefficients for the objective function (maximize)
num_constraints = 3
constraint_coeffs = [
    [3, 2, 1],  # Coefficients for constraint 1
    [1, 4, 3],  # Coefficients for constraint 2
    [1, 2, 2]   # Coefficients for constraint 3 (>= type)
]
constraint_types = ['<=', '<=', '>=']  # Types of constraints
b_values = [1200, 1500, 800]  # Right-hand side values
is_maximization = True  # This is a maximization problem

# Create the initial tableau
tableau, all_slack_vars, basis, coeffs = create_tableau_for_maximization(
    num_vars, objective_coeffs, num_constraints, constraint_coeffs, constraint_types, b_values, is_maximization
)

# Add all variable names to the list
all_vars = ['x1', 'x2', 'x3'] + all_slack_vars

# Identify the artificial variables in the tableau
artificial_vars = [i for i, var in enumerate(all_vars) if 'a' in var]

# Perform iterations using simplex_iteration
is_optimal = False
is_unbounded = False
iteration_count = 0

while not is_optimal and not is_unbounded:
    iteration_count += 1
    print(f"\nIteration {iteration_count}:")
    tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration(
        tableau, coeffs, basis, is_maximization, all_vars, artificial_vars
    )
    print("Updated Tableau:")
    print(tableau)
    print("Updated Basis:", [all_vars[i] for i in basis])

# Extract and display the optimal solution using our custom solution
if is_optimal:
    optimal_values, objective_value = extract_optimal_solution(tableau, basis, all_vars)
    print("\nCustom Simplex Optimal Solution:")
    for var, value in optimal_values.items():
        print(f"{var} = {value}")
    print(f"Optimal Value of the Objective Function: {objective_value}")

elif is_unbounded:
    print("The solution is unbounded.")

# Now, solve the same problem using PuLP and compare the results
print("\nSolving the problem with PuLP for comparison...")

# Define the problem in PuLP
prob = LpProblem("Reclitis_Hydropower_Optimization", LpMaximize if is_maximization else LpMinimize)

# Define decision variables
x_vars = [LpVariable(f"x{i+1}", lowBound=0) for i in range(num_vars)]

# Add the objective function
prob += sum(objective_coeffs[i] * x_vars[i] for i in range(num_vars)), "Objective"

# Add constraints
for i in range(num_constraints):
    if constraint_types[i] == '<=':
        prob += sum(constraint_coeffs[i][j] * x_vars[j] for j in range(num_vars)) <= b_values[i], f"Constraint_{i+1}"
    elif constraint_types[i] == '>=':
        prob += sum(constraint_coeffs[i][j] * x_vars[j] for j in range(num_vars)) >= b_values[i], f"Constraint_{i+1}"
    elif constraint_types[i] == '=':
        prob += sum(constraint_coeffs[i][j] * x_vars[j] for j in range(num_vars)) == b_values[i], f"Constraint_{i+1}"

# Solve the problem using PuLP
prob.solve()

# Display the PuLP results
print("\nPuLP Optimal Solution:")
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")
print(f"Optimal Value of the Objective Function: {pulp.value(prob.objective)}")

# Compare both solutions
print("\nComparison of Custom Simplex and PuLP Solutions:")
for i in range(num_vars):
    custom_value = optimal_values.get(f"x{i+1}", 0)
    pulp_value = x_vars[i].varValue
    print(f"x{i+1}: Custom = {custom_value}, PuLP = {pulp_value}")

custom_optimal_value = objective_value
pulp_optimal_value = value(prob.objective)
print(f"Optimal Value: Custom = {custom_optimal_value}, PuLP = {pulp_optimal_value}")

Initial Basis Indices: [3, 4, 6]
Initial Basis Variables and Their Values:
Variable s1 (Index 3): Value = 1200.0
Variable s2 (Index 4): Value = 1500.0
Variable a1 (Index 6): Value = -M

Iteration 1:
Updated Tableau:
[[ 2.50e+00  0.00e+00 -5.00e-01  1.00e+00 -5.00e-01  0.00e+00  0.00e+00
   4.50e+02]
 [ 2.50e-01  1.00e+00  7.50e-01  0.00e+00  2.50e-01  0.00e+00  0.00e+00
   3.75e+02]
 [ 5.00e-01  0.00e+00  5.00e-01  0.00e+00 -5.00e-01 -1.00e+00  1.00e+00
   5.00e+01]
 [ 4.96e+05  0.00e+00  4.97e+05  0.00e+00 -4.99e+05 -1.00e+06  0.00e+00
  -5.15e+07]]
Updated Basis: ['s1', 'x2', 'a1']

Iteration 2:
Updated Tableau:
[[ 3.00e+00  0.00e+00  0.00e+00  1.00e+00 -1.00e+00 -1.00e+00  1.00e+00
   5.00e+02]
 [-5.00e-01  1.00e+00  0.00e+00  0.00e+00  1.00e+00  1.50e+00 -1.50e+00
   3.00e+02]
 [ 1.00e+00  0.00e+00  1.00e+00  0.00e+00 -1.00e+00 -2.00e+00  2.00e+00
   1.00e+02]
 [-1.00e+03  0.00e+00  0.00e+00  0.00e+00 -2.00e+03 -6.00e+03 -9.94e+05
  -1.80e+06]]
Updated Basis: ['s1', 'x2', 'x3']

Cu

TypeError: 'int' object is not callable

In [ ]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 77.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
from pulp import LpMaximize, LpMinimize, LpProblem, LpVariable
import pulp

# Assuming all your custom simplex-related functions are defined...

# Define the problem and solve with the custom simplex method as shown in your example.

# Solving the problem with PuLP for comparison...
prob = LpProblem("Hydropower_Optimization", LpMinimize)

# Define the variables
x1 = LpVariable("x1", lowBound=0)
x2 = LpVariable("x2", lowBound=0)
x3 = LpVariable("x3", lowBound=0)

# Objective function
prob += 2000 * x1 + 1500 * x2 + 1000 * x3

# Constraints
prob += 6 * x1 + 2 * x2 + 4 * x3 >= 400
prob += 2 * x1 + 4 * x2 + 3 * x3 >= 375
prob += 5 * x1 + 2 * x2 + 1 * x3 >= 50

# Solve the problem using PuLP
prob.solve()

# Print PuLP results
print("\nPuLP Optimal Solution:")
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")
pulp_optimal_value = pulp.value(prob.objective)
print(f"Optimal Value of the Objective Function: {pulp_optimal_value}")

# Compare both solutions
print("\nComparison of Custom Simplex and PuLP Solutions:")
custom_optimal_value = objective_value
print(f"Optimal Value: Custom = {custom_optimal_value}, PuLP = {pulp_optimal_value}")



PuLP Optimal Solution:
x1 = 0.0
x2 = 0.0
x3 = 125.0
Optimal Value of the Objective Function: 125000.0

Comparison of Custom Simplex and PuLP Solutions:
Optimal Value: Custom = 125000.0, PuLP = 125000.0


In [ ]:
import numpy as np
from pulp import LpMaximize, LpMinimize, LpProblem, LpVariable
import pulp

# Assuming all your custom simplex-related functions are defined...

# Define the problem for the custom simplex solver
num_vars = 3
objective_coeffs = [2000, 1500, 1000]  # Coefficients for the objective function (minimize)
num_constraints = 3
constraint_coeffs = [
    [6, 2, 4],
    [2, 4, 3],
    [5, 2, 1]
]
constraint_types = ['>=', '>=', '>=']
b_values = [400, 375, 50]
is_maximization = False  # Set to False for a minimization problem

# Create the initial tableau for the custom simplex solver
tableau, all_slack_vars, basis, coeffs = create_tableau_for_maximization(
    num_vars, objective_coeffs, num_constraints, constraint_coeffs, constraint_types, b_values, is_maximization
)

# Add all variable names to the list
all_vars = ['x1', 'x2', 'x3'] + all_slack_vars

# Identify the artificial variables in the tableau
artificial_vars = [i for i, var in enumerate(all_vars) if 'a' in var]

# Perform iterations using the custom simplex method until the optimal solution is reached or the solution is found to be unbounded
is_optimal = False
is_unbounded = False
iteration_count = 0

# Iteration loop for custom simplex method
while not is_optimal and not is_unbounded:
    iteration_count += 1
    print(f"\nIteration {iteration_count}:")
    tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration(
        tableau, coeffs, basis, is_maximization, all_vars, artificial_vars
    )
    print("Updated Tableau:")
    for row in tableau:
        print([f"{val:.2f}" for val in row])
    print("Updated Basis:", [all_vars[i] for i in basis])

# Extract and print the final solution from the custom simplex solver
if is_optimal:
    optimal_solution, objective_value = extract_optimal_solution(tableau, basis, all_vars)
    print("\nCustom Simplex Optimal Solution:")
    for var, value in optimal_solution.items():
        print(f"{var} = {value}")
    print(f"Optimal Value of the Objective Function: {objective_value}")

# Solving the problem with PuLP for comparison
prob = LpProblem("Hydropower_Optimization", LpMinimize)

# Define the variables for PuLP
x1 = LpVariable("x1", lowBound=0)
x2 = LpVariable("x2", lowBound=0)
x3 = LpVariable("x3", lowBound=0)

# Objective function for PuLP
prob += 2000 * x1 + 1500 * x2 + 1000 * x3

# Constraints for PuLP
prob += 6 * x1 + 2 * x2 + 4 * x3 >= 400
prob += 2 * x1 + 4 * x2 + 3 * x3 >= 375
prob += 5 * x1 + 2 * x2 + 1 * x3 >= 50

# Solve the problem using PuLP
prob.solve()

# Print PuLP results
print("\nPuLP Optimal Solution:")
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")
pulp_optimal_value = pulp.value(prob.objective)
print(f"Optimal Value of the Objective Function: {pulp_optimal_value}")

# Compare both solutions
print("\nComparison of Custom Simplex and PuLP Solutions:")
for i in range(num_vars):
    custom_value = optimal_solution.get(f'x{i + 1}', 0) if is_optimal else "N/A"
    pulp_value = prob.variables()[i].varValue
    print(f"x{i + 1}: Custom = {custom_value}, PuLP = {pulp_value}")

print(f"Optimal Value: Custom = {objective_value if is_optimal else 'N/A'}, PuLP = {pulp_optimal_value}")

Initial Basis Indices: [6, 7, 8]
Initial Basis Variables and Their Values:
Variable a2 (Index 6): Value = M
Variable s3 (Index 7): Value = 375.0
Variable a3 (Index 8): Value = M

Iteration 1:
Updated Tableau:
['0.00', '-0.40', '2.80', '-1.00', '0.00', '1.20', '1.00', '0.00', '-1.20', '340.00']
['0.00', '3.20', '2.60', '0.00', '-1.00', '0.40', '0.00', '1.00', '-0.40', '355.00']
['1.00', '0.40', '0.20', '0.00', '0.00', '-0.20', '0.00', '0.00', '0.20', '10.00']
['0.00', '-2799300.00', '-5399400.00', '1000000.00', '1000000.00', '-1599600.00', '0.00', '0.00', '2599600.00', '695020000.00']
Updated Basis: ['a2', 's3', 'x1']

Iteration 2:
Updated Tableau:
['-14.00', '-6.00', '0.00', '-1.00', '0.00', '4.00', '1.00', '0.00', '-4.00', '200.00']
['-13.00', '-2.00', '0.00', '0.00', '-1.00', '3.00', '0.00', '1.00', '-3.00', '225.00']
['5.00', '2.00', '1.00', '0.00', '0.00', '-1.00', '0.00', '0.00', '1.00', '50.00']
['26997000.00', '7999500.00', '0.00', '1000000.00', '1000000.00', '-6999000.00', '0.0

In [ ]:
num_vars = 2
objective_coeffs = [10, 15]  # Coefficients for the objective function (maximize)
num_constraints = 3
constraint_coeffs = [
    [1, 1],  # Coefficients for constraint 1
    [1, 2],  # Coefficients for constraint 2
    [2, 1]   # Coefficients for constraint 3 (>= type)
]
constraint_types = ['>=', '<=', '<=']  # Types of constraints
b_values = [1, 6, 6]  # Right-hand side values
is_maximization = True  # This is a maximization problem

# Create the initial tableau
tableau, all_slack_vars, basis, coeffs = create_tableau_for_maximization(
    num_vars, objective_coeffs, num_constraints, constraint_coeffs, constraint_types, b_values, is_maximization
)

# Add all variable names to the list
all_vars = ['x1', 'x2'] + all_slack_vars

# Identify the artificial variables in the tableau
artificial_vars = [i for i, var in enumerate(all_vars) if 'a' in var]

# Display the initial tableau before starting the iterations
print("\nInitial Tableau:")
print(tableau)
print("Initial Basis Indices:", basis)
print("Initial Basis Variables and Their Values:")
for i in basis:
    print(f"Variable {all_vars[i]} (Index {i}): Value = {'M' if i >= num_vars + len(all_slack_vars) else tableau[i - len(basis), -1]}")

# Perform iterations using simplex_iteration
is_optimal = False
is_unbounded = False
iteration_count = 0

while not is_optimal and not is_unbounded:
    iteration_count += 1
    print(f"\nIteration {iteration_count}:")
    tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration(
        tableau, coeffs, basis, is_maximization, all_vars, artificial_vars
    )
    print("Updated Tableau:")
    print(tableau)
    print("Updated Basis:", [all_vars[i] for i in basis])

# Extract and display the optimal solution
if is_optimal:
    optimal_values, objective_value = extract_optimal_solution(tableau, basis, all_vars)
    print("\nOptimal Solution:")
    for var, value in optimal_values.items():
        print(f"{var} = {value}")
    print(f"Optimal Value of the Objective Function: {objective_value}")

elif is_unbounded:
    print("The solution is unbounded.")

Initial Basis Indices: [5, 3, 4]
Initial Basis Variables and Their Values:
Variable s3 (Index 5): Value = 1.0
Variable a1 (Index 3): Value = -M
Variable s2 (Index 4): Value = 6.0

Initial Tableau:
[[ 1.00000e+00  1.00000e+00 -1.00000e+00  0.00000e+00  0.00000e+00
   1.00000e+00  1.00000e+00]
 [ 1.00000e+00  2.00000e+00  0.00000e+00  1.00000e+00  0.00000e+00
   0.00000e+00  6.00000e+00]
 [ 2.00000e+00  1.00000e+00  0.00000e+00  0.00000e+00  1.00000e+00
   0.00000e+00  6.00000e+00]
 [ 9.99990e+05  9.99985e+05 -1.00000e+06  0.00000e+00  0.00000e+00
   0.00000e+00 -1.00000e+06]]
Initial Basis Indices: [5, 3, 4]
Initial Basis Variables and Their Values:
Variable s3 (Index 5): Value = 6.0
Variable a1 (Index 3): Value = 1.0
Variable s2 (Index 4): Value = 6.0

Iteration 1:
Updated Tableau:
[[ 1.0000e+00  1.0000e+00 -1.0000e+00  0.0000e+00  0.0000e+00  1.0000e+00
   1.0000e+00]
 [ 0.0000e+00  1.0000e+00  1.0000e+00  1.0000e+00  0.0000e+00 -1.0000e+00
   5.0000e+00]
 [ 0.0000e+00 -1.0000e+00  2.

In [ ]:
# Example usage of the updated function
num_vars = 2
objective_coeffs = [10, 15]  # Coefficients for the objective function (maximize)
num_constraints = 3
constraint_coeffs = [
    [1, 1],  # Coefficients for constraint 1
    [1, 2],  # Coefficients for constraint 2
    [2, 1]   # Coefficients for constraint 3 (>= type)
]
constraint_types = ['>=', '<=', '<=']  # Types of constraints
b_values = [1, 6, 6]  # Right-hand side values
is_maximization = True  # This is a maximization problem

# Create the initial tableau
tableau, all_slack_vars, basis, coeffs = create_tableau_for_maximization(
    num_vars, objective_coeffs, num_constraints, constraint_coeffs, constraint_types, b_values, is_maximization
)

# Add all variable names to the list
all_vars = ['x1', 'x2'] + all_slack_vars

# Identify the artificial variables in the tableau
artificial_vars = [i for i, var in enumerate(all_vars) if 'a' in var]

# Display the initial tableau before starting the iterations
print("\nInitial Tableau:")
print(tableau)
print("Initial Basis Indices:", basis)
print("Initial Basis Variables and Their Values:")
for i in basis:
    value = "M" if i >= num_vars + len(all_slack_vars) else tableau[i - len(basis), -1]
    print(f"Variable {all_vars[i]} (Index {i}): Value = {value}")

# Perform iterations using simplex_iteration_maximization
is_optimal = False
is_unbounded = False
iteration_count = 0

while not is_optimal and not is_unbounded:
    iteration_count += 1
    print(f"\nIteration {iteration_count}:")
    tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration_maximization(
        tableau, coeffs, basis, all_vars, artificial_vars
    )
    print("Updated Tableau:")
    print(tableau)
    print("Updated Basis Indices:", basis)
    print("Basis Variables and Their Values:")
    for i in basis:
        value = "M" if i >= num_vars + len(all_slack_vars) else tableau[i - len(basis), -1]
        print(f"Variable {all_vars[i]} (Index {i}): Value = {value}")

# Extract and display the optimal solution
if is_optimal:
    optimal_values, objective_value = extract_optimal_solution(tableau, basis, all_vars)
    print("\nOptimal Solution:")
    for var, value in optimal_values.items():
        print(f"{var} = {value}")
    print(f"Optimal Value of the Objective Function: {objective_value}")

elif is_unbounded:
    print("The solution is unbounded.")

Initial Basis Indices: [5, 3, 4]
Initial Basis Variables and Their Values:
Variable s3 (Index 5): Value = 1.0
Variable a1 (Index 3): Value = -M
Variable s2 (Index 4): Value = 6.0

Initial Tableau:
[[ 1.00000e+00  1.00000e+00 -1.00000e+00  0.00000e+00  0.00000e+00
   1.00000e+00  1.00000e+00]
 [ 1.00000e+00  2.00000e+00  0.00000e+00  1.00000e+00  0.00000e+00
   0.00000e+00  6.00000e+00]
 [ 2.00000e+00  1.00000e+00  0.00000e+00  0.00000e+00  1.00000e+00
   0.00000e+00  6.00000e+00]
 [ 9.99990e+05  9.99985e+05 -1.00000e+06  0.00000e+00  0.00000e+00
   0.00000e+00 -1.00000e+06]]
Initial Basis Indices: [5, 3, 4]
Initial Basis Variables and Their Values:
Variable s3 (Index 5): Value = 6.0
Variable a1 (Index 3): Value = 1.0
Variable s2 (Index 4): Value = 6.0

Iteration 1:


ValueError: not enough values to unpack (expected 5, got 4)

In [ ]:
# Example usage of the updated function

num_vars = 2
objective_coeffs = [10, 15]  # Coefficients for the objective function (maximize)
num_constraints = 3
constraint_coeffs = [
    [1, 1],  # Coefficients for constraint 1
    [1, 2],  # Coefficients for constraint 2
    [2, 1]   # Coefficients for constraint 3 (>= type)
]
constraint_types = ['>=', '<=', '<=']  # Types of constraints
b_values = [1, 6, 6]  # Right-hand side values
is_maximization = True  # This is a maximization problem

# Create the initial tableau
tableau, all_slack_vars, basis, coeffs = create_tableau_for_maximization(
    num_vars, objective_coeffs, num_constraints, constraint_coeffs, constraint_types, b_values, is_maximization
)

# Add all variable names to the list
all_vars = ['x1', 'x2'] + all_slack_vars

# Identify the artificial variables in the tableau
artificial_vars = [i for i, var in enumerate(all_vars) if 'a' in var]

# Display the initial tableau before starting the iterations
print("\nInitial Tableau:")
print(tableau)
print("Initial Basis Indices:", basis)
print("Initial Basis Variables and Their Values:")
for i in basis:
    value = "M" if i >= num_vars + len(all_slack_vars) else tableau[i - len(basis), -1]
    print(f"Variable {all_vars[i]} (Index {i}): Value = {value}")

# Perform iterations using simplex_iteration_maximization
is_optimal = False
is_unbounded = False
iteration_count = 0
max_iterations = 100  # Set a limit to prevent infinite looping

while not is_optimal and not is_unbounded and iteration_count < max_iterations:
    iteration_count += 1
    print(f"\nIteration {iteration_count}:")

    # Perform one iteration of the simplex method for maximization
    tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration_maximization(
        tableau, coeffs, basis, all_vars, artificial_vars
    )

    # Check if the artificial variable is still in the basis
    artificial_in_basis = any(basis_var in artificial_vars for basis_var in basis)

    # Display updated tableau and basis information after each iteration
    print("Updated Tableau:")
    print(tableau)
    print("Updated Basis Indices:", basis)
    print("Basis Variables and Their Values:")
    for idx, i in enumerate(basis):
        if i < len(all_vars):
            value = tableau[idx, -1]  # RHS value for the corresponding row
            print(f"Variable {all_vars[i]} (Index {i}): Value = {value}")

    # Check for optimality based on the objective row and artificial variables
    if all(coef <= 0 for coef in tableau[-1, :-1]) and all(rhs >= 0 for rhs in tableau[:-1, -1]) and not artificial_in_basis:
        is_optimal = True
    else:
        is_optimal = False

# Extract and display the optimal solution if found
if is_optimal:
    optimal_values, objective_value = extract_optimal_solution(tableau, basis, all_vars)
    print("\nOptimal Solution:")
    for var, value in optimal_values.items():
        print(f"{var} = {value}")
    print(f"Optimal Value of the Objective Function: {objective_value}")

elif is_unbounded:
    print("The solution is unbounded.")

elif iteration_count >= max_iterations:
    print("Reached the maximum number of iterations. The solution might not converge.")


Initial Basis Indices: [5, 3, 4]
Initial Basis Variables and Their Values:
Variable s3 (Index 5): Value = 1.0
Variable a1 (Index 3): Value = -M
Variable s2 (Index 4): Value = 6.0

Initial Tableau:
[[ 1.00000e+00  1.00000e+00 -1.00000e+00  0.00000e+00  0.00000e+00
   1.00000e+00  1.00000e+00]
 [ 1.00000e+00  2.00000e+00  0.00000e+00  1.00000e+00  0.00000e+00
   0.00000e+00  6.00000e+00]
 [ 2.00000e+00  1.00000e+00  0.00000e+00  0.00000e+00  1.00000e+00
   0.00000e+00  6.00000e+00]
 [ 9.99990e+05  9.99985e+05 -1.00000e+06  0.00000e+00  0.00000e+00
   0.00000e+00 -1.00000e+06]]
Initial Basis Indices: [5, 3, 4]
Initial Basis Variables and Their Values:
Variable s3 (Index 5): Value = 6.0
Variable a1 (Index 3): Value = 1.0
Variable s2 (Index 4): Value = 6.0

Iteration 1:
Updated Tableau:
[[ 1.0000e+00  1.0000e+00 -1.0000e+00  0.0000e+00  0.0000e+00  1.0000e+00
   1.0000e+00]
 [ 0.0000e+00  1.0000e+00  1.0000e+00  1.0000e+00  0.0000e+00 -1.0000e+00
   5.0000e+00]
 [ 0.0000e+00 -1.0000e+00  2.

In [ ]:
# Add all variable names to the list
all_vars = ['x1', 'x2'] + all_slack_vars

# Identify the artificial variables in the tableau
artificial_vars = [i for i, var in enumerate(all_vars) if 'a' in var]

# Display the initial tableau before starting the iterations
print("\nInitial Tableau:")
print(tableau)
print("Initial Basis Indices:", basis)

# Print the basis values to verify
print_basis_values(basis, tableau, all_vars, num_vars, len(all_slack_vars), artificial_vars, is_maximization)



Initial Tableau:
[[-4.33333333e+00 -6.66666667e-01  0.00000000e+00  0.00000000e+00
  -3.33333333e-01  1.00000000e+00  0.00000000e+00  3.33333333e-01
  -1.00000000e+00  7.50000000e+01]
 [-3.33333333e+00  3.33333333e+00  0.00000000e+00  1.00000000e+00
  -1.33333333e+00  0.00000000e+00 -1.00000000e+00  1.33333333e+00
   0.00000000e+00  1.00000000e+02]
 [ 6.66666667e-01  1.33333333e+00  1.00000000e+00  0.00000000e+00
  -3.33333333e-01  0.00000000e+00  0.00000000e+00  3.33333333e-01
   0.00000000e+00  1.25000000e+02]
 [ 1.33333333e+03  1.66666667e+02  0.00000000e+00  0.00000000e+00
   3.33333333e+02  0.00000000e+00  1.00000000e+06  9.99666667e+05
   1.00000000e+06  1.25000000e+05]]
Initial Basis Indices: [5, 3, 2]

Basis Variables and Their Values:
Variable a2 (Index 5): Value = M
Variable a1 (Index 3): Value = M
Variable s1 (Index 2): Value = 125.0


In [ ]:
import numpy as np

# Define the data
num_vars = 2
objective_coeffs = [10, 15]  # Coefficients for the objective function (maximize)
num_constraints = 3
constraint_coeffs = [
    [1, 1],  # Coefficients for constraint 1
    [1, 2],  # Coefficients for constraint 2
    [2, 1]   # Coefficients for constraint 3 (>= type)
]
constraint_types = ['>=', '<=', '<=']  # Types of constraints
b_values = [1, 6, 6]  # Right-hand side values
is_maximization = True  # This is a maximization problem

# Step 1: Create the initial tableau for maximization
tableau, all_vars, basis, coeffs = create_tableau_only_for_maximization(
    num_vars, objective_coeffs, num_constraints,
    constraint_coeffs, constraint_types, b_values
)

# Step 2: Iterate using the simplex method for maximization until an optimal solution is found
is_optimal = False
is_unbounded = False
iteration = 1

print("Initial Tableau:")
print(tableau)

while not is_optimal and not is_unbounded:
    print(f"\nIteration {iteration}:")
    tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration_maximization(
        tableau, coeffs, basis, all_vars, artificial_vars=[i for i, var in enumerate(all_vars) if var.startswith('a')]
    )
    print(tableau)
    iteration += 1

# Step 3: Output the final result
if is_optimal:
    print("\nOptimal Solution Reached")
    # Extract the solution
    solution = {}
    for i in range(num_vars):
        if i in basis:
            row = basis.index(i)
            solution[f"x{i+1}"] = tableau[row, -1]
        else:
            solution[f"x{i+1}"] = 0
    print("Solution:", solution)
    print("Optimal Objective Value:", tableau[-1, -1])
elif is_unbounded:
    print("\nThe solution is unbounded.")


Initial Basis Indices: [5, 3, 4]
Initial Basis Variables and Their Values:
Variable s3 (Index 5): Value = 1.0
Variable a1 (Index 3): Value = -M
Variable s2 (Index 4): Value = 6.0
Initial Tableau:
[[ 1.00000e+00  1.00000e+00 -1.00000e+00  0.00000e+00  0.00000e+00
   1.00000e+00  1.00000e+00]
 [ 1.00000e+00  2.00000e+00  0.00000e+00  1.00000e+00  0.00000e+00
   0.00000e+00  6.00000e+00]
 [ 2.00000e+00  1.00000e+00  0.00000e+00  0.00000e+00  1.00000e+00
   0.00000e+00  6.00000e+00]
 [ 9.99990e+05  9.99985e+05 -1.00000e+06  0.00000e+00  0.00000e+00
   0.00000e+00 -1.00000e+06]]

Iteration 1:
[[ 1.0000e+00  1.0000e+00 -1.0000e+00  0.0000e+00  0.0000e+00  1.0000e+00
   1.0000e+00]
 [ 0.0000e+00  1.0000e+00  1.0000e+00  1.0000e+00  0.0000e+00 -1.0000e+00
   5.0000e+00]
 [ 0.0000e+00 -1.0000e+00  2.0000e+00  0.0000e+00  1.0000e+00 -2.0000e+00
   4.0000e+00]
 [ 0.0000e+00 -5.0000e+00 -1.0000e+01  0.0000e+00  0.0000e+00 -9.9999e+05
  -1.0000e+01]]

Optimal Solution Reached
Solution: {'x1': 1.0, 

In [ ]:
import numpy as np

# Define the data for the problem
num_vars = 2
objective_coeffs = [10, 15]  # Coefficients for the objective function (maximize)
num_constraints = 3
constraint_coeffs = [
    [1, 1],  # Coefficients for constraint 1
    [1, 2],  # Coefficients for constraint 2
    [2, 1]   # Coefficients for constraint 3 (>= type)
]
constraint_types = ['>=', '<=', '<=']  # Types of constraints
b_values = [1, 6, 6]  # Right-hand side values
is_maximization = True  # This is a maximization problem

# Step 1: Create the initial tableau for maximization
tableau, all_vars, basis, coeffs = create_tableau_only_for_maximization(
    num_vars, objective_coeffs, num_constraints,
    constraint_coeffs, constraint_types, b_values
)

# Step 2: Iterate using the simplex method for maximization until an optimal solution is found
is_optimal = False
is_unbounded = False
iteration = 1

print("Initial Tableau:")
print(tableau)

while not is_optimal and not is_unbounded:
    print(f"\nIteration {iteration}:")
    try:
        tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration_maximization(
            tableau, coeffs, basis, all_vars, artificial_vars=[i for i, var in enumerate(all_vars) if var.startswith('a')]
        )
    except IndexError as e:
        print(f"Error during iteration: {e}")
        break
    print(tableau)
    iteration += 1

# Step 3: Output the final result
if is_optimal:
    print("\nOptimal Solution Reached")
    # Extract the solution
    solution = {}
    for i in range(num_vars):
        if i in basis:
            row = basis.index(i)
            solution[f"x{i+1}"] = tableau[row, -1]
        else:
            solution[f"x{i+1}"] = 0
    print("Solution:", solution)
    print("Optimal Objective Value:", tableau[-1, -1])
elif is_unbounded:
    print("\nThe solution is unbounded.")


Initial Basis Indices: [5, 3, 4]
Initial Basis Variables and Their Values:
Variable s3 (Index 5): Value = 1.0
Variable a1 (Index 3): Value = -M
Variable s2 (Index 4): Value = 6.0
Initial Tableau:
[[ 1.00000e+00  1.00000e+00 -1.00000e+00  0.00000e+00  0.00000e+00
   1.00000e+00  1.00000e+00]
 [ 1.00000e+00  2.00000e+00  0.00000e+00  1.00000e+00  0.00000e+00
   0.00000e+00  6.00000e+00]
 [ 2.00000e+00  1.00000e+00  0.00000e+00  0.00000e+00  1.00000e+00
   0.00000e+00  6.00000e+00]
 [ 9.99990e+05  9.99985e+05 -1.00000e+06  0.00000e+00  0.00000e+00
   0.00000e+00 -1.00000e+06]]

Iteration 1:
[[ 1.0000e+00  1.0000e+00 -1.0000e+00  0.0000e+00  0.0000e+00  1.0000e+00
   1.0000e+00]
 [ 0.0000e+00  1.0000e+00  1.0000e+00  1.0000e+00  0.0000e+00 -1.0000e+00
   5.0000e+00]
 [ 0.0000e+00 -1.0000e+00  2.0000e+00  0.0000e+00  1.0000e+00 -2.0000e+00
   4.0000e+00]
 [ 0.0000e+00 -5.0000e+00 -1.0000e+01  0.0000e+00  0.0000e+00 -9.9999e+05
  -1.0000e+01]]

Optimal Solution Reached
Solution: {'x1': 1.0, 

In [ ]:
import numpy as np

# Define the data
num_vars = 3
objective_coeffs = [5000, 4000, 6000]  # Coefficients for the objective function (maximize)
num_constraints = 3
constraint_coeffs = [
    [3, 2, 1],  # Coefficients for constraint 1
    [1, 4, 3],  # Coefficients for constraint 2
    [1, 2, 2]   # Coefficients for constraint 3 (>= type)
]
constraint_types = ['<=', '<=', '>=']  # Types of constraints
b_values = [1200, 1500, 800]  # Right-hand side values
is_maximization = True  # This is a maximization problem

# Step 1: Create the initial tableau for maximization
tableau, all_vars, basis, coeffs = create_tableau_only_for_maximization(
    num_vars, objective_coeffs, num_constraints,
    constraint_coeffs, constraint_types, b_values
)

# Step 2: Iterate using the simplex method for maximization until an optimal solution is found
is_optimal = False
is_unbounded = False
iteration = 1

print("Initial Tableau:")
print(tableau)

while not is_optimal and not is_unbounded:
    print(f"\nIteration {iteration}:")
    tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration_maximization(
        tableau, coeffs, basis, all_vars, artificial_vars=[i for i, var in enumerate(all_vars) if var.startswith('a')]
    )
    print(tableau)
    iteration += 1

# Step 3: Output the final result
if is_optimal:
    print("\nOptimal Solution Reached")
    # Extract the solution
    solution = {}
    for i in range(num_vars):
        if i in basis:
            row = basis.index(i)
            solution[f"x{i+1}"] = tableau[row, -1]
        else:
            solution[f"x{i+1}"] = 0
    print("Solution:", solution)
    print("Optimal Objective Value:", tableau[-1, -1])
elif is_unbounded:
    print("\nThe solution is unbounded.")


Initial Basis Indices: [3, 4, 6]
Initial Basis Variables and Their Values:
Variable s1 (Index 3): Value = 1200.0
Variable s2 (Index 4): Value = 1500.0
Variable a1 (Index 6): Value = -M
Initial Tableau:
[[ 3.000e+00  2.000e+00  1.000e+00  1.000e+00  0.000e+00  0.000e+00
   0.000e+00  1.200e+03]
 [ 1.000e+00  4.000e+00  3.000e+00  0.000e+00  1.000e+00  0.000e+00
   0.000e+00  1.500e+03]
 [ 1.000e+00  2.000e+00  2.000e+00  0.000e+00  0.000e+00 -1.000e+00
   1.000e+00  8.000e+02]
 [ 9.950e+05  1.996e+06  1.994e+06  0.000e+00  0.000e+00 -1.000e+06
   0.000e+00 -8.000e+08]]

Iteration 1:
[[ 2.50e+00  0.00e+00 -5.00e-01  1.00e+00 -5.00e-01  0.00e+00  0.00e+00
   4.50e+02]
 [ 2.50e-01  1.00e+00  7.50e-01  0.00e+00  2.50e-01  0.00e+00  0.00e+00
   3.75e+02]
 [ 5.00e-01  0.00e+00  5.00e-01  0.00e+00 -5.00e-01 -1.00e+00  1.00e+00
   5.00e+01]
 [ 4.96e+05  0.00e+00  4.97e+05  0.00e+00 -4.99e+05 -1.00e+06  0.00e+00
  -5.15e+07]]

Iteration 2:
[[ 3.00e+00  0.00e+00  0.00e+00  1.00e+00 -1.00e+00 -1.0

In [ ]:
import numpy as np
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Define the data
num_vars = 3
objective_coeffs = [5000, 4000, 6000]  # Coefficients for the objective function (maximize)
num_constraints = 3
constraint_coeffs = [
    [3, 2, 1],  # Coefficients for constraint 1
    [1, 4, 3],  # Coefficients for constraint 2
    [1, 2, 2]   # Coefficients for constraint 3 (>= type)
]
constraint_types = ['<=', '<=', '>=']  # Types of constraints
b_values = [1200, 1500, 800]  # Right-hand side values
is_maximization = True  # This is a maximization problem

# Step 1: Create the initial tableau for maximization
tableau, all_vars, basis, coeffs = create_tableau_only_for_maximization(
    num_vars, objective_coeffs, num_constraints,
    constraint_coeffs, constraint_types, b_values
)

# Step 2: Iterate using the simplex method for maximization until an optimal solution is found
is_optimal = False
is_unbounded = False
iteration = 1

print("Initial Tableau:")
print(tableau)

while not is_optimal and not is_unbounded:
    print(f"\nIteration {iteration}:")
    tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration_maximization(
        tableau, coeffs, basis, all_vars, artificial_vars=[i for i, var in enumerate(all_vars) if var.startswith('a')]
    )
    print(tableau)
    iteration += 1

# Step 3: Output the final result from the custom simplex method
if is_optimal:
    print("\nOptimal Solution Reached (Custom Simplex Method)")
    # Extract the solution
    solution = {}
    for i in range(num_vars):
        if i in basis:
            row = basis.index(i)
            solution[f"x{i+1}"] = tableau[row, -1]
        else:
            solution[f"x{i+1}"] = 0
    print("Solution:", solution)
    print("Optimal Objective Value:", tableau[-1, -1])
elif is_unbounded:
    print("\nThe solution is unbounded.")

# Step 4: Solve the same problem using PuLP for comparison
# Create the problem
prob = LpProblem("Maximization_Problem", LpMaximize)

# Create variables
x = [LpVariable(f"x{i+1}", lowBound=0) for i in range(num_vars)]

# Set the objective function
prob += lpSum(objective_coeffs[i] * x[i] for i in range(num_vars)), "Objective"

# Add the constraints
for i in range(num_constraints):
    if constraint_types[i] == '<=':
        prob += lpSum(constraint_coeffs[i][j] * x[j] for j in range(num_vars)) <= b_values[i], f"Constraint_{i+1}"
    elif constraint_types[i] == '>=':
        prob += lpSum(constraint_coeffs[i][j] * x[j] for j in range(num_vars)) >= b_values[i], f"Constraint_{i+1}"
    elif constraint_types[i] == '=':
        prob += lpSum(constraint_coeffs[i][j] * x[j] for j in range(num_vars)) == b_values[i], f"Constraint_{i+1}"

# Solve the problem
prob.solve()

# Output the results from PuLP
print("\nOptimal Solution Reached (PuLP)")
pulp_solution = {f"x{i+1}": x[i].varValue for i in range(num_vars)}
print("Solution:", pulp_solution)
print("Optimal Objective Value:", prob.objective.value())


Initial Basis Indices: [3, 4, 6]
Initial Basis Variables and Their Values:
Variable s1 (Index 3): Value = 1200.0
Variable s2 (Index 4): Value = 1500.0
Variable a1 (Index 6): Value = -M
Initial Tableau:
[[ 3.000e+00  2.000e+00  1.000e+00  1.000e+00  0.000e+00  0.000e+00
   0.000e+00  1.200e+03]
 [ 1.000e+00  4.000e+00  3.000e+00  0.000e+00  1.000e+00  0.000e+00
   0.000e+00  1.500e+03]
 [ 1.000e+00  2.000e+00  2.000e+00  0.000e+00  0.000e+00 -1.000e+00
   1.000e+00  8.000e+02]
 [ 9.950e+05  1.996e+06  1.994e+06  0.000e+00  0.000e+00 -1.000e+06
   0.000e+00 -8.000e+08]]

Iteration 1:
[[ 2.50e+00  0.00e+00 -5.00e-01  1.00e+00 -5.00e-01  0.00e+00  0.00e+00
   4.50e+02]
 [ 2.50e-01  1.00e+00  7.50e-01  0.00e+00  2.50e-01  0.00e+00  0.00e+00
   3.75e+02]
 [ 5.00e-01  0.00e+00  5.00e-01  0.00e+00 -5.00e-01 -1.00e+00  1.00e+00
   5.00e+01]
 [ 4.96e+05  0.00e+00  4.97e+05  0.00e+00 -4.99e+05 -1.00e+06  0.00e+00
  -5.15e+07]]

Iteration 2:
[[ 3.00e+00  0.00e+00  0.00e+00  1.00e+00 -1.00e+00 -1.0

In [ ]:
import numpy as np
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Define the data
num_vars = 2
objective_coeffs = [2, 4]  # Coefficients for the objective function (maximize)
num_constraints = 3
constraint_coeffs = [
    [2, 1],  # Coefficients for constraint 1
    [3, 2],  # Coefficients for constraint 2
    [1, 1]   # Coefficients for constraint 3 (>= type)
]
constraint_types = ['<=', '>=', '=']  # Types of constraints
b_values = [18, 30, 26]  # Right-hand side values
is_maximization = True  # This is a maximization problem

# Correct the tracking and updating of the basis and coefficients during simplex iterations.

# Step 1: Create the initial tableau for maximization
tableau, all_vars, basis, coeffs = create_tableau_only_for_maximization(
    num_vars, objective_coeffs, num_constraints,
    constraint_coeffs, constraint_types, b_values
)

# Step 2: Iterate using the simplex method for maximization until an optimal solution is found
is_optimal = False
is_unbounded = False
iteration = 1

print("Initial Tableau:")
print(tableau)
print(f"Initial Coefficients: {coeffs}")

while not is_optimal and not is_unbounded:
    print(f"\nIteration {iteration}:")
    # Perform a single iteration of the simplex method
    try:
        tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration_maximization(
            tableau, coeffs, basis, all_vars, artificial_vars=[i for i, var in enumerate(all_vars) if var.startswith('a')]
        )
    except IndexError as e:
        print(f"Error: Basis index out of range. Current Basis: {basis}")
        print("Error details:", e)
        break

    # Print tableau and basis information after each iteration
    print("Tableau after iteration:")
    print(tableau)

    # Ensure basis is within range and valid
    try:
        current_basis_vars = [all_vars[basis[i]] for i in range(len(basis)) if basis[i] < len(all_vars)]
        print(f"Basis Variables: {current_basis_vars}")
    except IndexError:
        print(f"Error: Basis index out of range. Current Basis: {basis}")

    # Print current coefficients
    print(f"Coefficients: {coeffs}\n")

    iteration += 1

# Step 3: Output the final result from the custom simplex method
if is_optimal:
    print("\nOptimal Solution Reached (Custom Simplex Method)")
    # Extract the solution
    solution = {}
    for i in range(num_vars):
        if i in basis:
            row = basis.index(i)
            solution[f"x{i+1}"] = tableau[row, -1]
        else:
            solution[f"x{i+1}"] = 0
    print("Solution:", solution)
    print("Optimal Objective Value:", tableau[-1, -1])
elif is_unbounded:
    print("\nThe solution is unbounded.")

# Step 4: Solve the same problem using PuLP for comparison
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Create the problem
prob = LpProblem("Maximization_Problem", LpMaximize)

# Create variables
x = [LpVariable(f"x{i+1}", lowBound=0) for i in range(num_vars)]

# Set the objective function
prob += lpSum(objective_coeffs[i] * x[i] for i in range(num_vars)), "Objective"

# Add the constraints
for i in range(num_constraints):
    if constraint_types[i] == '<=':
        prob += lpSum(constraint_coeffs[i][j] * x[j] for j in range(num_vars)) <= b_values[i], f"Constraint_{i+1}"
    elif constraint_types[i] == '>=':
        prob += lpSum(constraint_coeffs[i][j] * x[j] for j in range(num_vars)) >= b_values[i], f"Constraint_{i+1}"
    elif constraint_types[i] == '=':
        prob += lpSum(constraint_coeffs[i][j] * x[j] for j in range(num_vars)) == b_values[i], f"Constraint_{i+1}"

# Solve the problem
prob.solve()

# Output the results from PuLP
print("\nOptimal Solution Reached (PuLP)")
pulp_solution = {f"x{i+1}": x[i].varValue for i in range(num_vars)}
print("Solution:", pulp_solution)
print("Optimal Objective Value:", prob.objective.value())


Initial Basis Indices: [2, 4, 5]
Initial Basis Variables and Their Values:
Variable s1 (Index 2): Value = 18.0
Variable a1 (Index 4): Value = -M
Variable a2 (Index 5): Value = -M
Initial Tableau:
[[ 2.000000e+00  1.000000e+00  1.000000e+00  0.000000e+00  0.000000e+00
   0.000000e+00  1.800000e+01]
 [ 3.000000e+00  2.000000e+00  0.000000e+00 -1.000000e+00  1.000000e+00
   0.000000e+00  3.000000e+01]
 [ 1.000000e+00  1.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
   1.000000e+00  2.600000e+01]
 [ 3.999998e+06  2.999996e+06  0.000000e+00 -1.000000e+06  0.000000e+00
   0.000000e+00 -5.600000e+07]]
Initial Coefficients: [-2, -4, 0, 0, -1000000.0, -1000000.0]

Iteration 1:
Tableau after iteration:
[[ 1.0000000e+00  5.0000000e-01  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00  9.0000000e+00]
 [ 0.0000000e+00  5.0000000e-01  0.0000000e+00 -1.0000000e+00
   1.0000000e+00  0.0000000e+00  3.0000000e+00]
 [ 0.0000000e+00  5.0000000e-01  0.0000000e+00  0.0000000e+00
   0.0000

In [ ]:
import numpy as np
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Define the data
num_vars = 2
objective_coeffs = [2000, 1500]
num_constraints = 3
constraint_coeffs = [[6, 2], [2, 4], [4, 12]]
constraint_types = ['>=', '>=', '>=']
b_values = [8, 12, 24]
is_maximization = True  # This is a maximization problem

# Step 1: Create the initial tableau for maximization
tableau, all_slack_vars, basis, coeffs = create_tableau_only_for_maximization(
    num_vars, objective_coeffs, num_constraints,
    constraint_coeffs, constraint_types, b_values
)

# Step 1.1: Create all_vars list properly to include decision, slack, and artificial variables
all_vars = [f"x{i+1}" for i in range(num_vars)] + all_slack_vars

# Add artificial variables to the list if they exist in the tableau
num_artificial_vars = tableau.shape[1] - 1 - len(all_vars)
all_vars += [f"a{i+1}" for i in range(num_artificial_vars)]

# Validate all_vars list length matches the tableau columns (excluding RHS column)
assert len(all_vars) == tableau.shape[1] - 1, "Length of all_vars should match the number of columns in the tableau (excluding RHS)."

# Correct the initial basis values: Set slack variables to 0, artificial variables to -M
M = 1e6  # Big M value
basis_values = []
for idx in basis:
    if idx >= len(all_vars):
        raise IndexError(f"Basis index {idx} is out of range for all_vars with length {len(all_vars)}")

    var_name = all_vars[idx]
    if 's' in var_name:  # Slack variable
        basis_values.append(0)
    elif 'a' in var_name:  # Artificial variable
        basis_values.append(-M)
    else:
        raise ValueError(f"Unexpected variable in basis: {var_name}")

# Correct coefficients: decision variables get their objective function values, slack = 0, artificial = -M
coeffs = [
    objective_coeffs[i] if i < num_vars else 0 if 's' in all_vars[i] else -M
    for i in range(len(all_vars))
]

# Display the initial tableau
print("\nInitial Tableau:")
print(tableau)
print("Initial Basis Indices:", basis)
print("Initial Basis Variables and Their Values:")
for row_idx, var_idx in enumerate(basis):
    if var_idx >= len(all_vars):
        raise IndexError(f"Basis index {var_idx} is out of range for all_vars with length {len(all_vars)}")

    var_name = all_vars[var_idx]
    value = -M if var_idx in [idx for idx, var in enumerate(all_vars) if var.startswith('a')] else tableau[row_idx, -1]
    print(f"Variable {var_name} (Index {var_idx}): Value = {value}")
print(f"Initial Coefficients: {coeffs}")

# Step 2: Perform iterations until the optimal solution is reached or unbounded
is_optimal = False
is_unbounded = False
iteration_count = 0

while not is_optimal and not is_unbounded:
    iteration_count += 1
    print(f"\nIteration {iteration_count}:")

    # Perform a single iteration of the simplex method for maximization
    try:
        tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration_maximization(
            tableau, coeffs, basis, all_vars, artificial_vars=[i for i, var in enumerate(all_vars) if var.startswith('a')]
        )
    except IndexError as e:
        print(f"Error: Basis index out of range. Current Basis: {basis}")
        print("Error details:", e)
        break

    # Print the updated tableau and basis information after each iteration
    print("Tableau after iteration:")
    print(tableau)
    print(f"Updated Basis Variables: {[all_vars[i] for i in basis]}")
    print(f"Updated Coefficients: {coeffs}\n")

# Step 3: Output the final result from the custom simplex method
if is_optimal:
    print("\nOptimal Solution Reached (Custom Simplex Method)")
    # Extract the solution
    solution = {}
    for i in range(num_vars):
        if i in basis:
            row = basis.index(i)
            solution[f"x{i+1}"] = tableau[row, -1]
        else:
            solution[f"x{i+1}"] = 0
    print("Solution:", solution)
    print("Optimal Objective Value:", tableau[-1, -1])
elif is_unbounded:
    print("\nThe solution is unbounded.")

# Step 4: Solve the same problem using PuLP for comparison
# Create the problem using PuLP
prob = LpProblem("Maximization_Problem", LpMaximize)

# Create variables
x = [LpVariable(f"x{i+1}", lowBound=0) for i in range(num_vars)]

# Set the objective function
prob += lpSum(objective_coeffs[i] * x[i] for i in range(num_vars)), "Objective"

# Add the constraints
for i in range(num_constraints):
    if constraint_types[i] == '>=':
        prob += lpSum(constraint_coeffs[i][j] * x[j] for j in range(num_vars)) >= b_values[i], f"Constraint_{i+1}"
    elif constraint_types[i] == '<=':
        prob += lpSum(constraint_coeffs[i][j] * x[j] for j in range(num_vars)) <= b_values[i], f"Constraint_{i+1}"
    elif constraint_types[i] == '=':
        prob += lpSum(constraint_coeffs[i][j] * x[j] for j in range(num_vars)) == b_values[i], f"Constraint_{i+1}"

# Solve the problem
prob.solve()

# Output the results from PuLP
print("\nOptimal Solution Reached (PuLP)")
pulp_solution = {f"x{i+1}": x[i].varValue for i in range(num_vars)}
print("Solution:", pulp_solution)
print("Optimal Objective Value:", prob.objective.value())


Initial Basis Indices: [5, 6, 7]
Initial Basis Variables and Their Values:
Variable a2 (Index 5): Value = -M
Variable s3 (Index 6): Value = 12.0
Variable a3 (Index 7): Value = -M

Initial Tableau:
[[ 6.00000e+00  2.00000e+00 -1.00000e+00  0.00000e+00  0.00000e+00
   1.00000e+00  0.00000e+00  0.00000e+00  8.00000e+00]
 [ 2.00000e+00  4.00000e+00  0.00000e+00 -1.00000e+00  0.00000e+00
   0.00000e+00  1.00000e+00  0.00000e+00  1.20000e+01]
 [ 4.00000e+00  1.20000e+01  0.00000e+00  0.00000e+00 -1.00000e+00
   0.00000e+00  0.00000e+00  1.00000e+00  2.40000e+01]
 [ 1.19980e+07  1.79985e+07 -1.00000e+06 -1.00000e+06 -1.00000e+06
   0.00000e+00  0.00000e+00  0.00000e+00 -4.40000e+07]]
Initial Basis Indices: [5, 6, 7]
Initial Basis Variables and Their Values:
Variable a2 (Index 5): Value = -1000000.0
Variable s3 (Index 6): Value = 12.0
Variable a3 (Index 7): Value = -1000000.0
Initial Coefficients: [2000, 1500, 0, -1000000.0, 0, -1000000.0, 0, -1000000.0]

Iteration 1:
Tableau after iteration:


In [ ]:
import numpy as np
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Define the data
num_vars = 2
objective_coeffs = [2, 4]
num_constraints = 3
constraint_coeffs = [
    [2, 1],
    [3, 2],
    [1, 2]
]
constraint_types = ['<=', '>=', '=']
b_values = [18, 30, 26]

# Create the initial tableau for maximization
tableau, all_slack_vars, basis, coeffs = create_tableau_only_for_maximization(
    num_vars, objective_coeffs, num_constraints,
    constraint_coeffs, constraint_types, b_values
)

# Step 1: Create the list of all variables
all_vars = [f"x{i+1}" for i in range(num_vars)] + all_slack_vars
num_artificial_vars = tableau.shape[1] - 1 - len(all_vars)
all_vars += [f"a{i+1}" for i in range(num_artificial_vars)]

# Correctly calculate the initial basis values
M = 1e6
for i, idx in enumerate(basis):
    var_name = all_vars[idx]
    if 's' in var_name:
        tableau[i, -1]  # Make sure the RHS values are correct in the tableau
    elif 'a' in var_name:
        # No need to explicitly set artificial variables to -M here,
        # as it's already handled in create_tableau_only_for_maximization
        pass

# Correct the initial coefficients in the objective row
coeffs = [
    objective_coeffs[i] if i < num_vars else 0 if 's' in all_vars[i] else -M
    for i in range(len(all_vars))
]

# Correct calculation of the initial objective row
for col_idx in range(len(all_vars)):
    original_coeff = coeffs[col_idx]
    weighted_sum = 0
    for i, basic_var_index in enumerate(basis):
        weighted_sum += coeffs[basic_var_index] * tableau[i, col_idx]
    tableau[-1, col_idx] = original_coeff - weighted_sum

# Correct calculation of the RHS value for the objective row
rhs_value = 0
for i, basic_var_index in enumerate(basis):
    rhs_value += coeffs[basic_var_index] * tableau[i, -1]
tableau[-1, -1] = rhs_value

# Display the initial tableau
print("\nInitial Tableau:")
print(tableau)
print("Initial Basis Indices:", basis)
print("Initial Basis Variables and Their Values:")
for row_idx, var_idx in enumerate(basis):
    var_name = all_vars[var_idx]
    value = -M if var_idx in [idx for idx, var in enumerate(all_vars) if var.startswith('a')] else tableau[row_idx, -1]
    print(f"Variable {var_name} (Index {var_idx}): Value = {value}")
print(f"Initial Coefficients: {coeffs}")

# Perform simplex iterations until an optimal solution is reached or the solution is found to be unbounded
is_optimal = False
is_unbounded = False
iteration_count = 0

while not is_optimal and not is_unbounded:
    iteration_count += 1
    print(f"\nIteration {iteration_count}:")

    # Identify the entering column (most positive coefficient for maximization)
    entering_column = identify_entering_column_maximization(tableau)
    if entering_column is None:
        is_optimal = True
        break

    # Identify the leaving row using key ratios
    key_ratios = []
    for i in range(len(basis)):
        rhs_value = tableau[i, -1]
        entering_value = tableau[i, entering_column]
        if entering_value > 0:
            key_ratios.append(rhs_value / entering_value)
        else:
            key_ratios.append(np.inf)

    # Output key ratios for tracking
    print(f"Key Ratios for Leaving Variable Selection: {key_ratios}")

    # Determine the row that will leave the basis
    leaving_row = identify_leaving_row_maximization(tableau, entering_column)
    if leaving_row is None:
        is_unbounded = True
        break

    # Perform the pivot operation for the identified entering and leaving variables
    tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration_maximization(
        tableau, coeffs, basis, all_vars, artificial_vars=[i for i, var in enumerate(all_vars) if var.startswith('a')]
    )

    # Display the updated tableau
    print("Tableau after iteration:")
    print(tableau)
    print(f"Updated Basis Variables: {[all_vars[i] for i in basis]}")
    print(f"Updated Coefficients: {coeffs}\n")

# Output the final result from the custom simplex method
if is_optimal:
    print("\nOptimal Solution Reached (Custom Simplex Method)")
    # Extract the solution
    solution = {}
    for i in range(num_vars):
        if i in basis:
            row = basis.index(i)
            solution[f"x{i+1}"] = tableau[row, -1]
        else:
            solution[f"x{i+1}"] = 0
    print("Solution:", solution)
    print("Optimal Objective Value:", tableau[-1, -1])
elif is_unbounded:
    print("\nThe solution is unbounded.")


Initial Basis Indices: [2, 4, 5]
Initial Basis Variables and Their Values:
Variable s1 (Index 2): Value = 0
Variable a1 (Index 4): Value = -M
Variable a2 (Index 5): Value = -M

Initial Tableau:
[[ 2.000000e+00  1.000000e+00  1.000000e+00  0.000000e+00  0.000000e+00
   0.000000e+00  1.800000e+01]
 [ 3.000000e+00  2.000000e+00  0.000000e+00 -1.000000e+00  1.000000e+00
   0.000000e+00  3.000000e+01]
 [ 1.000000e+00  2.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
   1.000000e+00  2.600000e+01]
 [ 4.000002e+06  4.000004e+06  0.000000e+00 -1.000000e+06  0.000000e+00
   0.000000e+00 -5.600000e+07]]
Initial Basis Indices: [2, 4, 5]
Initial Basis Variables and Their Values:
Variable s1 (Index 2): Value = 18.0
Variable a1 (Index 4): Value = -1000000.0
Variable a2 (Index 5): Value = -1000000.0
Initial Coefficients: [2, 4, 0, 0, -1000000.0, -1000000.0]

Iteration 1:
Key Ratios for Leaving Variable Selection: [18.0, 15.0, 13.0]
Tableau after iteration:
[[ 1.500000e+00  0.000000e+00  1.00000

In [ ]:
import numpy as np
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Define the data
num_vars = 2
objective_coeffs = [10, 15]
num_constraints = 3
constraint_coeffs = [
    [1, 1],  # Coefficients for constraint 1
    [1, 2],  # Coefficients for constraint 2
    [2, 1]   # Coefficients for constraint 3 (>= type)
]
constraint_types = ['>=', '<=', '<=']
b_values = [1, 6, 6]

# Create the initial tableau for maximization
tableau, all_slack_vars, basis, coeffs = create_tableau_only_for_maximization(
    num_vars, objective_coeffs, num_constraints,
    constraint_coeffs, constraint_types, b_values
)

# Step 1: Create the list of all variables
all_vars = [f"x{i+1}" for i in range(num_vars)] + all_slack_vars
num_artificial_vars = tableau.shape[1] - 1 - len(all_vars)
all_vars += [f"a{i+1}" for i in range(num_artificial_vars)]

# Correctly calculate the initial basis values
M = 1e6
for i, idx in enumerate(basis):
    var_name = all_vars[idx]
    if 's' in var_name:
        tableau[i, -1]  # Make sure the RHS values are correct in the tableau
    elif 'a' in var_name:
        # No need to explicitly set artificial variables to -M here,
        # as it's already handled in create_tableau_only_for_maximization
        pass

# Correct the initial coefficients in the objective row
coeffs = [
    objective_coeffs[i] if i < num_vars else 0 if 's' in all_vars[i] else -M
    for i in range(len(all_vars))
]

# Correct calculation of the initial objective row
for col_idx in range(len(all_vars)):
    original_coeff = coeffs[col_idx]
    weighted_sum = 0
    for i, basic_var_index in enumerate(basis):
        weighted_sum += coeffs[basic_var_index] * tableau[i, col_idx]
    tableau[-1, col_idx] = original_coeff - weighted_sum

# Correct calculation of the RHS value for the objective row
rhs_value = 0
for i, basic_var_index in enumerate(basis):
    rhs_value += coeffs[basic_var_index] * tableau[i, -1]
tableau[-1, -1] = rhs_value

# Display the initial tableau
print("\nInitial Tableau:")
print(tableau)
print("Initial Basis Indices:", basis)
print("Initial Basis Variables and Their Values:")
for row_idx, var_idx in enumerate(basis):
    var_name = all_vars[var_idx]
    value = -M if var_idx in [idx for idx, var in enumerate(all_vars) if var.startswith('a')] else tableau[row_idx, -1]
    print(f"Variable {var_name} (Index {var_idx}): Value = {value}")
print(f"Initial Coefficients: {coeffs}")

# Perform simplex iterations until an optimal solution is reached or the solution is found to be unbounded
is_optimal = False
is_unbounded = False
iteration_count = 0

while not is_optimal and not is_unbounded:
    iteration_count += 1
    print(f"\nIteration {iteration_count}:")

    # Identify the entering column (most positive coefficient for maximization)
    entering_column = identify_entering_column_maximization(tableau)
    if entering_column is None:
        is_optimal = True
        break

    # Identify the leaving row using key ratios
    key_ratios = []
    for i in range(len(basis)):
        rhs_value = tableau[i, -1]
        entering_value = tableau[i, entering_column]
        if entering_value > 0:
            key_ratios.append(rhs_value / entering_value)
        else:
            key_ratios.append(np.inf)

    # Output key ratios for tracking
    print(f"Key Ratios for Leaving Variable Selection: {key_ratios}")

    # Determine the row that will leave the basis
    leaving_row = identify_leaving_row_maximization(tableau, entering_column)
    if leaving_row is None:
        is_unbounded = True
        break

    # Perform the pivot operation for the identified entering and leaving variables
    tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration_maximization(
        tableau, coeffs, basis, all_vars, artificial_vars=[i for i, var in enumerate(all_vars) if var.startswith('a')]
    )

    # Display the updated tableau
    print("Tableau after iteration:")
    print(tableau)
    print(f"Updated Basis Variables: {[all_vars[i] for i in basis]}")
    print(f"Updated Coefficients: {coeffs}\n")

# Output the final result from the custom simplex method
if is_optimal:
    print("\nOptimal Solution Reached (Custom Simplex Method)")
    # Extract the solution
    solution = {}
    for i in range(num_vars):
        if i in basis:
            row = basis.index(i)
            solution[f"x{i+1}"] = tableau[row, -1]
        else:
            solution[f"x{i+1}"] = 0
    print("Solution:", solution)
    print("Optimal Objective Value:", tableau[-1, -1])
elif is_unbounded:
    print("\nThe solution is unbounded.")


Initial Basis Indices: [5, 3, 4]
Initial Basis Variables and Their Values:
Variable s3 (Index 5): Value = 0
Variable a1 (Index 3): Value = -M
Variable s2 (Index 4): Value = 0

Initial Tableau:
[[ 1.000000e+00  1.000000e+00 -1.000000e+00  0.000000e+00  0.000000e+00
   1.000000e+00  1.000000e+00]
 [ 1.000000e+00  2.000000e+00  0.000000e+00  1.000000e+00  0.000000e+00
   0.000000e+00  6.000000e+00]
 [ 2.000000e+00  1.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00
   0.000000e+00  6.000000e+00]
 [ 1.000010e+06  2.000015e+06  0.000000e+00  0.000000e+00  0.000000e+00
   0.000000e+00 -6.000000e+06]]
Initial Basis Indices: [5, 3, 4]
Initial Basis Variables and Their Values:
Variable s3 (Index 5): Value = 1.0
Variable a1 (Index 3): Value = -1000000.0
Variable s2 (Index 4): Value = 6.0
Initial Coefficients: [10, 15, 0, -1000000.0, 0, 0]

Iteration 1:
Key Ratios for Leaving Variable Selection: [1.0, 3.0, 6.0]
Tableau after iteration:
[[ 1.000000e+00  1.000000e+00 -1.000000e+00  0.000000e+00

In [ ]:
num_vars = 2
objective_coeffs = [10, 15]  # Coefficients for the objective function (maximize)
num_constraints = 3
constraint_coeffs = [
    [1, 1],  # Coefficients for constraint 1
    [1, 2],  # Coefficients for constraint 2
    [2, 1]   # Coefficients for constraint 3 (>= type)
]
constraint_types = ['>=', '<=', '<=']  # Types of constraints
b_values = [1, 6, 6]  # Right-hand side values
is_maximization = True

In [ ]:
import numpy as np
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Define the data
num_vars = 2
objective_coeffs = [10, 15]
num_constraints = 3
constraint_coeffs = [
    [1, 1],  # Coefficients for constraint 1
    [1, 2],  # Coefficients for constraint 2
    [2, 1]   # Coefficients for constraint 3 (>= type)
]
constraint_types = ['>=', '<=', '<=']
b_values = [1, 6, 6]

# Create the initial tableau for maximization
tableau, all_slack_vars, basis, coeffs = create_tableau_only_for_maximization(
    num_vars, objective_coeffs, num_constraints,
    constraint_coeffs, constraint_types, b_values
)

# Step 1: Create the list of all variables
all_vars = [f"x{i+1}" for i in range(num_vars)] + all_slack_vars
num_artificial_vars = tableau.shape[1] - 1 - len(all_vars)
all_vars += [f"a{i+1}" for i in range(num_artificial_vars)]

# Correctly calculate the initial basis values
M = 1e6
for i, idx in enumerate(basis):
    var_name = all_vars[idx]
    if 's' in var_name:
        tableau[i, -1]  # Make sure the RHS values are correct in the tableau
    elif 'a' in var_name:
        # No need to explicitly set artificial variables to -M here,
        # as it's already handled in create_tableau_only_for_maximization
        pass

# Correct the initial coefficients in the objective row
coeffs = [
    objective_coeffs[i] if i < num_vars else 0 if 's' in all_vars[i] else -M
    for i in range(len(all_vars))
]

# Correct calculation of the initial objective row
for col_idx in range(len(all_vars)):
    original_coeff = coeffs[col_idx]
    weighted_sum = 0
    for i, basic_var_index in enumerate(basis):
        weighted_sum += coeffs[basic_var_index] * tableau[i, col_idx]
    tableau[-1, col_idx] = original_coeff - weighted_sum

# Correct calculation of the RHS value for the objective row
rhs_value = 0
for i, basic_var_index in enumerate(basis):
    rhs_value += coeffs[basic_var_index] * tableau[i, -1]
tableau[-1, -1] = rhs_value

# Display the initial tableau
print("\nInitial Tableau:")
print(tableau)
print("Initial Basis Indices:", basis)
print("Initial Basis Variables and Their Values:")
for row_idx, var_idx in enumerate(basis):
    var_name = all_vars[var_idx]
    value = -M if var_idx in [idx for idx, var in enumerate(all_vars) if var.startswith('a')] else tableau[row_idx, -1]
    print(f"Variable {var_name} (Index {var_idx}): Value = {value}")
print(f"Initial Coefficients: {coeffs}")

# Perform simplex iterations until an optimal solution is reached or the solution is found to be unbounded
is_optimal = False
is_unbounded = False
iteration_count = 0

while not is_optimal and not is_unbounded:
    iteration_count += 1
    print(f"\nIteration {iteration_count}:")

    # Identify the entering column (most positive coefficient for maximization)
    entering_column = identify_entering_column_maximization(tableau)
    if entering_column is None:
        is_optimal = True
        break

    # Identify the leaving row using key ratios
    key_ratios = []
    for i in range(len(basis)):
        rhs_value = tableau[i, -1]
        entering_value = tableau[i, entering_column]
        if entering_value > 0:
            key_ratios.append(rhs_value / entering_value)
        else:
            key_ratios.append(np.inf)

    # Output key ratios for tracking
    print(f"Key Ratios for Leaving Variable Selection: {key_ratios}")

    # Determine the row that will leave the basis
    leaving_row = identify_leaving_row_maximization(tableau, entering_column)
    if leaving_row is None:
        is_unbounded = True
        break

    # Perform the pivot operation for the identified entering and leaving variables
    tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration_maximization(
        tableau, coeffs, basis, all_vars, artificial_vars=[i for i, var in enumerate(all_vars) if var.startswith('a')]
    )

    # Display the updated tableau
    print("Tableau after iteration:")
    print(tableau)
    print(f"Updated Basis Variables: {[all_vars[i] for i in basis]}")
    print(f"Updated Coefficients: {coeffs}\n")

# Output the final result from the custom simplex method
if is_optimal:
    print("\nOptimal Solution Reached (Custom Simplex Method)")
    # Extract the solution
    solution = {}
    for i in range(num_vars):
        if i in basis:
            row = basis.index(i)
            solution[f"x{i+1}"] = tableau[row, -1]
        else:
            solution[f"x{i+1}"] = 0
    print("Solution:", solution)
    print("Optimal Objective Value:", tableau[-1, -1])
# Solve with Pulp to compare
    model = LpProblem(name="Maximization_Problem", sense=LpMaximize)
    x = {i: LpVariable(name=f"x{i+1}", lowBound=0) for i in range(num_vars)}
    model += lpSum(objective_coeffs[i] * x[i] for i in range(num_vars))
    for i in range(num_constraints):
        if constraint_types[i] == '<=':
            model += lpSum(constraint_coeffs[i][j] * x[j] for j in range(num_vars)) <= b_values[i]
        elif constraint_types[i] == '>=':
            model += lpSum(constraint_coeffs[i][j] * x[j] for j in range(num_vars)) >= b_values[i]
        elif constraint_types[i] == '=':
            model += lpSum(constraint_coeffs[i][j] * x[j] for j in range(num_vars)) == b_values[i]

    model.solve()

    print("\nOptimal Solution Reached (Pulp)")
    for var in x.values():
        print(f"{var.name}: {var.value()}")
    print("Objective Value:", model.objective.value())

elif is_unbounded:
    print("\nThe solution is unbounded.")

Initial Basis Indices: [5, 3, 4]
Initial Basis Variables and Their Values:
Variable s3 (Index 5): Value = 0
Variable a1 (Index 3): Value = -M
Variable s2 (Index 4): Value = 0

Initial Tableau:
[[       1.        1.       -1.        0.        0.        1.        1.]
 [       1.        2.        0.        1.        0.        0.        6.]
 [       2.        1.        0.        0.        1.        0.        6.]
 [ 1000010.  2000015.        0.        0.        0.        0. -6000000.]]
Initial Basis Indices: [5, 3, 4]
Initial Basis Variables and Their Values:
Variable s3 (Index 5): Value = 1.0
Variable a1 (Index 3): Value = -1000000.0
Variable s2 (Index 4): Value = 6.0
Initial Coefficients: [10, 15, 0, -1000000.0, 0, 0]

Iteration 1:
Key Ratios for Leaving Variable Selection: [1.0, 3.0, 6.0]
Tableau after iteration:
[[       1.        1.       -1.        0.        0.        1.        1.]
 [      -1.        0.        2.        1.        0.       -2.        4.]
 [       1.        0.        1.

In [ ]:
import numpy as np
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Define the data
num_vars = 3
objective_coeffs = [5000, 4000, 6000]
num_constraints = 3
constraint_coeffs = [
    [3, 2, 1],  # Coefficients for constraint 1
    [1, 4, 3],  # Coefficients for constraint 2
    [1, 2, 2]   # Coefficients for constraint 3 (>= type)
]
constraint_types = ['<=', '<=', '>=']  # Types of constraints
b_values = [1200, 1500, 800]

# Create the initial tableau for maximization
tableau, all_slack_vars, basis, coeffs = create_tableau_only_for_maximization(
    num_vars, objective_coeffs, num_constraints,
    constraint_coeffs, constraint_types, b_values
)

# Step 1: Create the list of all variables
all_vars = [f"x{i+1}" for i in range(num_vars)] + all_slack_vars
num_artificial_vars = tableau.shape[1] - 1 - len(all_vars)
all_vars += [f"a{i+1}" for i in range(num_artificial_vars)]

# Correctly calculate the initial basis values
M = 1e6
for i, idx in enumerate(basis):
    var_name = all_vars[idx]
    if 's' in var_name:
        tableau[i, -1]  # Make sure the RHS values are correct in the tableau
    elif 'a' in var_name:
        # No need to explicitly set artificial variables to -M here,
        # as it's already handled in create_tableau_only_for_maximization
        pass

# Correct the initial coefficients in the objective row
coeffs = [
    objective_coeffs[i] if i < num_vars else 0 if 's' in all_vars[i] else -M
    for i in range(len(all_vars))
]

# Correct calculation of the initial objective row
for col_idx in range(len(all_vars)):
    original_coeff = coeffs[col_idx]
    weighted_sum = 0
    for i, basic_var_index in enumerate(basis):
        weighted_sum += coeffs[basic_var_index] * tableau[i, col_idx]
    tableau[-1, col_idx] = original_coeff - weighted_sum

# Correct calculation of the RHS value for the objective row
rhs_value = 0
for i, basic_var_index in enumerate(basis):
    rhs_value += coeffs[basic_var_index] * tableau[i, -1]
tableau[-1, -1] = rhs_value

# Display the initial tableau
print("\nInitial Tableau:")
print(tableau)
print("Initial Basis Indices:", basis)
print("Initial Basis Variables and Their Values:")
for row_idx, var_idx in enumerate(basis):
    var_name = all_vars[var_idx]
    value = -M if var_idx in [idx for idx, var in enumerate(all_vars) if var.startswith('a')] else tableau[row_idx, -1]
    print(f"Variable {var_name} (Index {var_idx}): Value = {value}")
print(f"Initial Coefficients: {coeffs}")

# Perform simplex iterations until an optimal solution is reached or the solution is found to be unbounded
is_optimal = False
is_unbounded = False
iteration_count = 0

while not is_optimal and not is_unbounded:
    iteration_count += 1
    print(f"\nIteration {iteration_count}:")

    # Identify the entering column (most positive coefficient for maximization)
    entering_column = identify_entering_column_maximization(tableau)
    if entering_column is None:
        is_optimal = True
        break

    # Identify the leaving row using key ratios
    key_ratios = []
    for i in range(len(basis)):
        rhs_value = tableau[i, -1]
        entering_value = tableau[i, entering_column]
        if entering_value > 0:
            key_ratios.append(rhs_value / entering_value)
        else:
            key_ratios.append(np.inf)

    # Output key ratios for tracking
    print(f"Key Ratios for Leaving Variable Selection: {key_ratios}")

    # Determine the row that will leave the basis
    leaving_row = identify_leaving_row_maximization(tableau, entering_column)
    if leaving_row is None:
        is_unbounded = True
        break

    # Perform the pivot operation for the identified entering and leaving variables
    tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration_maximization(
        tableau, coeffs, basis, all_vars, artificial_vars=[i for i, var in enumerate(all_vars) if var.startswith('a')]
    )

    # Display the updated tableau
    print("Tableau after iteration:")
    print(tableau)
    print(f"Updated Basis Variables: {[all_vars[i] for i in basis]}")
    print(f"Updated Coefficients: {coeffs}\n")

# Output the final result from the custom simplex method
if is_optimal:
    print("\nOptimal Solution Reached (Custom Simplex Method)")
    # Extract the solution
    solution = {}
    for i in range(num_vars):
        if i in basis:
            row = basis.index(i)
            solution[f"x{i+1}"] = tableau[row, -1]
        else:
            solution[f"x{i+1}"] = 0
    print("Solution:", solution)
    print("Optimal Objective Value:", tableau[-1, -1])
# Solve with Pulp to compare
    model = LpProblem(name="Maximization_Problem", sense=LpMaximize)
    x = {i: LpVariable(name=f"x{i+1}", lowBound=0) for i in range(num_vars)}
    model += lpSum(objective_coeffs[i] * x[i] for i in range(num_vars))
    for i in range(num_constraints):
        if constraint_types[i] == '<=':
            model += lpSum(constraint_coeffs[i][j] * x[j] for j in range(num_vars)) <= b_values[i]
        elif constraint_types[i] == '>=':
            model += lpSum(constraint_coeffs[i][j] * x[j] for j in range(num_vars)) >= b_values[i]
        elif constraint_types[i] == '=':
            model += lpSum(constraint_coeffs[i][j] * x[j] for j in range(num_vars)) == b_values[i]

    model.solve()

    print("\nOptimal Solution Reached (Pulp)")
    for var in x.values():
        print(f"{var.name}: {var.value()}")
    print("Objective Value:", model.objective.value())

elif is_unbounded:
    print("\nThe solution is unbounded.")

Initial Basis Indices: [3, 4, 6]
Initial Basis Variables and Their Values:
Variable s1 (Index 3): Value = 0
Variable s2 (Index 4): Value = 0
Variable a1 (Index 6): Value = -M

Initial Tableau:
[[ 3.000e+00  2.000e+00  1.000e+00  1.000e+00  0.000e+00  0.000e+00  0.000e+00  1.200e+03]
 [ 1.000e+00  4.000e+00  3.000e+00  0.000e+00  1.000e+00  0.000e+00  0.000e+00  1.500e+03]
 [ 1.000e+00  2.000e+00  2.000e+00  0.000e+00  0.000e+00 -1.000e+00  1.000e+00  8.000e+02]
 [ 1.005e+06  2.004e+06  2.006e+06  0.000e+00  0.000e+00 -1.000e+06  0.000e+00 -8.000e+08]]
Initial Basis Indices: [3, 4, 6]
Initial Basis Variables and Their Values:
Variable s1 (Index 3): Value = 1200.0
Variable s2 (Index 4): Value = 1500.0
Variable a1 (Index 6): Value = -1000000.0
Initial Coefficients: [5000, 4000, 6000, 0, 0, 0, -1000000.0]

Iteration 1:
Key Ratios for Leaving Variable Selection: [1200.0, 500.0, 400.0]
Tableau after iteration:
[[      2.5       1.        0.        1.        0.        0.5       0.      800. ]

num_vars = 3
objective_coeffs = [5000, 4000, 6000]  # Coefficients for the objective function (maximize)
num_constraints = 3
constraint_coeffs = [
    [3, 2, 1],  # Coefficients for constraint 1
    [1, 4, 3],  # Coefficients for constraint 2
    [1, 2, 2]   # Coefficients for constraint 3 (>= type)
]
constraint_types = ['<=', '<=', '>=']  # Types of constraints
b_values = [1200, 1500, 800]

In [ ]:
import numpy as np
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Define the data
num_vars = 5
objective_coeffs = [2, 1, -1, 3, -1]
num_constraints = 3
constraint_coeffs = [
    [3, 0, 2, 0, -1],  # Coefficients for constraint 1
    [1, -1, 1, 0, 0],  # Coefficients for constraint 2
    [1, 0, 1, 1, 0]   # Coefficients for constraint 3 (>= type)
]
constraint_types = ['=', '=', '=']  # Types of constraints
b_values = [12, 5, 6]

# Create the initial tableau for maximization
tableau, all_slack_vars, basis, coeffs = create_tableau_only_for_maximization(
    num_vars, objective_coeffs, num_constraints,
    constraint_coeffs, constraint_types, b_values
)

# Step 1: Create the list of all variables
all_vars = [f"x{i+1}" for i in range(num_vars)] + all_slack_vars
num_artificial_vars = tableau.shape[1] - 1 - len(all_vars)
all_vars += [f"a{i+1}" for i in range(num_artificial_vars)]

# Correctly calculate the initial basis values
M = 1e6
for i, idx in enumerate(basis):
    var_name = all_vars[idx]
    if 's' in var_name:
        tableau[i, -1]  # Make sure the RHS values are correct in the tableau
    elif 'a' in var_name:
        # No need to explicitly set artificial variables to -M here,
        # as it's already handled in create_tableau_only_for_maximization
        pass

# Correct the initial coefficients in the objective row
coeffs = [
    objective_coeffs[i] if i < num_vars else 0 if 's' in all_vars[i] else -M
    for i in range(len(all_vars))
]

# Correct calculation of the initial objective row
for col_idx in range(len(all_vars)):
    original_coeff = coeffs[col_idx]
    weighted_sum = 0
    for i, basic_var_index in enumerate(basis):
        weighted_sum += coeffs[basic_var_index] * tableau[i, col_idx]
    tableau[-1, col_idx] = original_coeff - weighted_sum

# Correct calculation of the RHS value for the objective row
rhs_value = 0
for i, basic_var_index in enumerate(basis):
    rhs_value += coeffs[basic_var_index] * tableau[i, -1]
tableau[-1, -1] = rhs_value

# Display the initial tableau
print("\nInitial Tableau:")
print(tableau)
print("Initial Basis Indices:", basis)
print("Initial Basis Variables and Their Values:")
for row_idx, var_idx in enumerate(basis):
    var_name = all_vars[var_idx]
    value = -M if var_idx in [idx for idx, var in enumerate(all_vars) if var.startswith('a')] else tableau[row_idx, -1]
    print(f"Variable {var_name} (Index {var_idx}): Value = {value}")
print(f"Initial Coefficients: {coeffs}")

# Perform simplex iterations until an optimal solution is reached or the solution is found to be unbounded
is_optimal = False
is_unbounded = False
iteration_count = 0

while not is_optimal and not is_unbounded:
    iteration_count += 1
    print(f"\nIteration {iteration_count}:")

    # Identify the entering column (most positive coefficient for maximization)
    entering_column = identify_entering_column_maximization(tableau)
    if entering_column is None:
        is_optimal = True
        break

    # Identify the leaving row using key ratios
    key_ratios = []
    for i in range(len(basis)):
        rhs_value = tableau[i, -1]
        entering_value = tableau[i, entering_column]
        if entering_value > 0:
            key_ratios.append(rhs_value / entering_value)
        else:
            key_ratios.append(np.inf)

    # Output key ratios for tracking
    print(f"Key Ratios for Leaving Variable Selection: {key_ratios}")

    # Determine the row that will leave the basis
    leaving_row = identify_leaving_row_maximization(tableau, entering_column)
    if leaving_row is None:
        is_unbounded = True
        break

    # Perform the pivot operation for the identified entering and leaving variables
    tableau, basis, all_vars, is_optimal, is_unbounded = simplex_iteration_maximization(
        tableau, coeffs, basis, all_vars, artificial_vars=[i for i, var in enumerate(all_vars) if var.startswith('a')]
    )

    # Display the updated tableau
    print("Tableau after iteration:")
    print(tableau)
    print(f"Updated Basis Variables: {[all_vars[i] for i in basis]}")
    print(f"Updated Coefficients: {coeffs}\n")

# Output the final result from the custom simplex method
if is_optimal:
    print("\nOptimal Solution Reached (Custom Simplex Method)")
    # Extract the solution
    solution = {}
    for i in range(num_vars):
        if i in basis:
            row = basis.index(i)
            solution[f"x{i+1}"] = tableau[row, -1]
        else:
            solution[f"x{i+1}"] = 0
    print("Solution:", solution)
    print("Optimal Objective Value:", tableau[-1, -1])
# Solve with Pulp to compare
    model = LpProblem(name="Maximization_Problem", sense=LpMaximize)
    x = {i: LpVariable(name=f"x{i+1}", lowBound=0) for i in range(num_vars)}
    model += lpSum(objective_coeffs[i] * x[i] for i in range(num_vars))
    for i in range(num_constraints):
        if constraint_types[i] == '<=':
            model += lpSum(constraint_coeffs[i][j] * x[j] for j in range(num_vars)) <= b_values[i]
        elif constraint_types[i] == '>=':
            model += lpSum(constraint_coeffs[i][j] * x[j] for j in range(num_vars)) >= b_values[i]
        elif constraint_types[i] == '=':
            model += lpSum(constraint_coeffs[i][j] * x[j] for j in range(num_vars)) == b_values[i]

    model.solve()

    print("\nOptimal Solution Reached (Pulp)")
    for var in x.values():
        print(f"{var.name}: {var.value()}")
    print("Objective Value:", model.objective.value())

elif is_unbounded:
    print("\nThe solution is unbounded.")

Initial Basis Indices: [5, 6, 7]
Initial Basis Variables and Their Values:
Variable a1 (Index 5): Value = -M
Variable a2 (Index 6): Value = -M
Variable a3 (Index 7): Value = -M

Initial Tableau:
[[        3.         0.         2.         0.        -1.         1.         0.         0.
         12.]
 [        1.        -1.         1.         0.         0.         0.         1.         0.
          5.]
 [        1.         0.         1.         1.         0.         0.         0.         1.
          6.]
 [  5000002.   -999999.   3999999.   1000003.  -1000001.         0.         0.         0.
  -23000000.]]
Initial Basis Indices: [5, 6, 7]
Initial Basis Variables and Their Values:
Variable a1 (Index 5): Value = -1000000.0
Variable a2 (Index 6): Value = -1000000.0
Variable a3 (Index 7): Value = -1000000.0
Initial Coefficients: [2, 1, -1, 3, -1, -1000000.0, -1000000.0, -1000000.0]

Iteration 1:
Key Ratios for Leaving Variable Selection: [4.0, 5.0, 6.0]
Tableau after iteration:
[[       1.  